# 컴피티션 링크
- https://www.kaggle.com/t/2e45abe9f1434b59a3358365432a48bb

# 고객별 백화점 구매기록 데이터셋
- 미혼(1) or 기혼(0) 고객을 예측
- 학습용 구매기록 데이터
    - https://drive.google.com/file/d/1tg41qXu02FK55bFa8P1Gx2URhwPUCoQQ/view?usp=sharing
- 학습용 정답 데이터
    - https://drive.google.com/file/d/1-9AcU9nAoO4SzSmqdCOYTvWNpA5Pdt3q/view?usp=sharing
- 테스트용 구매기록 데이터
    - https://drive.google.com/file/d/1-AwDfGlHm9rNtpnHIWOK96jBJYy3f2SZ/view?usp=sharing
- 제출 양식 데이터
    - https://drive.google.com/file/d/1-Qv7SlsY5Eu3bRR7Z0IUJyLkOO1Fl3y6/view?usp=sharing

In [1]:
# %pip install --upgrade pandas

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


- 데이터 경로 변수

In [3]:
DATA_PATH = "/content/drive/MyDrive/data/"
DATA_PATH

'/content/drive/MyDrive/data/'

- 데이터 불러오기

In [4]:
import pandas as pd
import numpy as np

train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터

train_tr.shape , train_target.shape , test_tr.shape , submit.shape

((523105, 7), (14940, 2), (441196, 7), (12225, 2))

In [5]:
train_tr[train_tr['대분류'] == '공산품']

,ID,구매일시,지점코드,대분류,중분류,브랜드코드,구매가격
2,train_7200,2004-05-01 10:20:00,A112000,공산품,용기보증,5100,34937
6,train_12777,2004-05-01 10:30:00,A112000,공산품,일용잡화,5100,17000
9,train_35,2004-05-01 10:40:00,A112000,공산품,용기보증,5100,28358
14,train_14716,2004-05-01 10:40:00,A373000,공산품,상품군미지정,5100,8100
15,train_7407,2004-05-01 10:40:00,A112000,공산품,야채,5100,9560
...,...,...,...,...,...,...,...
523054,train_10849,2005-04-29 19:20:00,A202000,공산품,냉장식품,5100,3320
523079,train_5332,2005-04-29 19:30:00,A202000,공산품,과자,5100,27000
523093,train_5332,2005-04-29 19:40:00,A202000,공산품,가정잡화,5100,19778
523096,train_13390,2005-04-29 19:50:00,A112000,공산품,용기보증,5100,91826


In [6]:
# from IPython.display import display, HTML
# br_html = HTML("<br><br>")
# display(HTML("<h1>학습용 구매기록 데이터</h1>"), train_tr.head(), br_html)
# display(HTML("<h1>학습용 정답 데이터</h1>"), train_target.head(), br_html)
# display(HTML("<h1>테스트용 구매기록 데이터</h1>"), test_tr.head(), br_html)
# display(HTML("<h1>제출 양식 데이터</h1>"), submit.head(), br_html)

In [7]:
# display(HTML("<h1>학습용 구매기록 데이터</h1>"))
# display(train_tr.info(), br_html)

# display(HTML("<h1>학습용 정답 데이터</h1>"))
# display(train_target.info(), br_html)

# display(HTML("<h1>테스트용 구매기록 데이터</h1>"))
# display(test_tr.info(), br_html)

# display(HTML("<h1>제출 양식 데이터</h1>"))
# display(submit.info(), br_html)

In [8]:
# pd.concat([train_tr, train_target], axis = 1)

# 특성 공학(Feature Engineering)

## 날짜 형식으로 변환

In [9]:
# train_tr["구매일시"] = pd.to_datetime(train_tr["구매일시"])
# test_tr["구매일시"] = pd.to_datetime(test_tr["구매일시"])
# train_tr["구매일시"].max()

## 새로 만든 feature와 병합할 고객ID로만 이루어진 데이터프레임 생성

In [10]:
# train_ft = train_target[["ID"]]
# test_ft = submit[["ID"]]

# train_ft.shape, test_ft.shape

In [11]:
# train_tr['중분류'].values[:]

## 구매일시를 이용한 특성생성

In [12]:
# 저녁 시간18~21, 5월4일~5월5일, 어버이날

In [13]:
# agg_list = [
#         # 컬럼명, 집계 방식
#         ('내점일수','nunique'),
#         ('구매주기', lambda x: int( (x.max() - x.min()).days / x.dt.date.nunique()) ),
#         ('주말방문비율', lambda x: np.mean(x.dt.weekday>4)),
#         ('봄_구매비율', lambda x: np.mean(x.dt.month.isin([3,4,5]))),
#         ('여름_구매비율', lambda x: np.mean(x.dt.month.isin([6,7,8]))),
#         ('가을_구매비율', lambda x: np.mean(x.dt.month.isin([9,10,11]))),
#         ('겨울_구매비율', lambda x: np.mean(x.dt.month.isin([1,2,12]))),
#         ('주구매요일', lambda x: x.dt.weekday.mode()[0]),
#         ('일별평균구매건수', lambda x:  x.count() / x.dt.date.nunique() ),
#         ('거래개월수', lambda x: x.dt.date.astype(str).str[:-3].nunique() ),
#     ]

# tmp = train_tr.groupby('ID')["구매일시"].agg(agg_list).reset_index()
# train_ft = train_ft.merge(tmp, how='left',on="ID")
# train_ft.head()

In [14]:
# tmp = test_tr.groupby('ID')["구매일시"].agg(agg_list).reset_index()
# test_ft = test_ft.merge(tmp, how='left',on="ID")
# test_ft.head()

## 지점을 이용한 특성생성

In [15]:
# agg_list = [
#           ("방문지점수","nunique"),
#           ('주구매지점', lambda x: x.mode()[0]),
#     ]

# tmp = train_tr.groupby('ID')["지점코드"].agg(agg_list).reset_index()
# train_ft = train_ft.merge(tmp, how='left',on="ID")
# train_ft.head()

In [16]:
# tmp = test_tr.groupby('ID')["지점코드"].agg(agg_list).reset_index()
# test_ft = test_ft.merge(tmp, how='left',on="ID")
# test_ft.head()

## 브랜드코드를 이용한 특성생성

In [17]:
# agg_list = [
#              ('브랜드코드_nunique', 'nunique'),
#              ]

# tmp = train_tr.groupby('ID')["브랜드코드"].agg(agg_list).reset_index()
# train_ft = train_ft.merge(tmp, how='left',on="ID")
# train_ft.head()

In [18]:
# tmp = test_tr.groupby('ID')["브랜드코드"].agg(agg_list).reset_index()
# test_ft = test_ft.merge(tmp, how='left',on="ID")
# test_ft.head()

## 중분류를 이용한 특성생성

In [19]:
# agg_list = [
#             ('중분류_nunique', 'nunique'),
#             ('주구매_중분류', lambda x: x.mode()[0]),
#             ]
# tmp = train_tr.groupby('ID')["중분류"].agg(agg_list).reset_index()
# train_ft = train_ft.merge(tmp, how='left')
# train_ft.head()

In [20]:
# tmp = test_tr.groupby('ID')["중분류"].agg(agg_list).reset_index()
# test_ft = test_ft.merge(tmp, how='left')
# test_ft.head()

## 대분류를 이용한 특성생성

In [21]:
# agg_list = [
#             ('대분류_nunique', 'nunique'),
#             ('주구매_대분류', lambda x: x.mode()[0]),
#             ]

# tmp = train_tr.groupby('ID')["대분류"].agg(agg_list).reset_index()
# train_ft = train_ft.merge(tmp, how='left')
# train_ft.head()

In [22]:
# tmp = test_tr.groupby('ID')["대분류"].agg(agg_list).reset_index()
# test_ft = test_ft.merge(tmp, how='left')
# test_ft.head()

## 구매가격을 이용한 특성생성

In [23]:
# agg_list = [
#         ('총구매액','sum'),
#         ('구매건수', 'count'),
#         ('평균구매액', "mean"),
#         ('최대구매액', 'max'),
#         ('총구매액_왜도', 'skew'),
#         ('최소구매액',lambda x: x[x > 0].min() ) ,
#         ('환불금액',lambda x: x[x < 0].sum() ) ,
#         ('환불건수', lambda x: ( x < 0 ).sum() ),
#         ('구매금액표준편차',lambda x: x[x>0].std() )
#     ]

# tmp = train_tr.groupby('ID')["구매가격"].agg(agg_list).reset_index()
# train_ft = train_ft.merge(tmp, how='left')
# train_ft.head()

In [24]:
# tmp = test_tr.groupby('ID')["구매가격"].agg(agg_list).reset_index()
# test_ft = test_ft.merge(tmp, how='left')
# test_ft.head()

In [25]:
# train_ft.shape, test_ft.shape

## pivot_table을 이용한 특성 생성

In [26]:
# train_tmp = pd.pivot_table(train_tr,index="ID",columns="중분류",values="구매가격",aggfunc="count",fill_value=0).add_prefix("pivot_cnt_").reset_index()
# train_ft = train_ft.merge(train_tmp,how="left",on="ID")
# train_ft.head()

In [27]:
# test_tmp = pd.pivot_table(test_tr,index="ID",columns="중분류",values="구매가격",aggfunc="count",fill_value=0).add_prefix("pivot_cnt_").reset_index()
# test_tmp.head()

In [28]:
# for col in train_tmp.columns:
#     if col not in test_tmp.columns:
#         test_tmp[col] = 0

# test_tmp = test_tmp[train_tmp.columns]

In [29]:
# test_ft = test_ft.merge(test_tmp,how="left",on="ID")
# test_ft.head()

In [30]:
# cols = [ col for col in train_ft.columns if col.startswith("pivot_cnt_") ]

In [31]:
# train_ft["중분류별_구매횟수_std"] = train_ft[cols].std(axis=1)
# train_ft["중분류별_구매횟수_skew"] = train_ft[cols].skew(axis=1)
# train_ft["중분류별_구매횟수_kurt"] = train_ft[cols].kurt(axis=1)

# test_ft["중분류별_구매횟수_std"] = test_ft[cols].std(axis=1)
# test_ft["중분류별_구매횟수_skew"] = test_ft[cols].skew(axis=1)
# test_ft["중분류별_구매횟수_kurt"] = test_ft[cols].kurt(axis=1)

# train_ft.shape, test_ft.shape

In [32]:
final_project = '/content/drive/MyDrive/멋사_final_project/'

In [33]:
train_ft = pd.read_csv(f"{final_project}train_baseline.csv") # 특성추출_베이스라인 train 결과 불러오기
test_ft = pd.read_csv(f"{final_project}test_baseline.csv") # 특성추출_베이스라인 test 결과 불러오기


train_ft.shape, test_ft.shape

((14940, 332), (12225, 332))

## 구매일시 활용

In [34]:
# 저녁 시간18~21, 5월4일~5월5일(어린이날), 5월8일(어버이날)

In [35]:
train_tr["구매일시"] = pd.to_datetime(train_tr["구매일시"], format='%Y-%m-%d %H:%M:%S')
test_tr["구매일시"] = pd.to_datetime(test_tr["구매일시"], format='%Y-%m-%d %H:%M:%S')

In [36]:
train_tr['구매일시'].dtype

dtype('<M8[ns]')

In [37]:
train_tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 523105 entries, 0 to 523104
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   ID      523105 non-null  object        
 1   구매일시    523105 non-null  datetime64[ns]
 2   지점코드    523105 non-null  object        
 3   대분류     523105 non-null  object        
 4   중분류     523105 non-null  object        
 5   브랜드코드   523105 non-null  int64         
 6   구매가격    523105 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 27.9+ MB


In [38]:
train_tr['구매일시'].isnull().sum()

0

In [39]:
train_tr['구매일시'].head()

,구매일시
0,2004-05-01 09:40:00
1,2004-05-01 09:40:00
2,2004-05-01 10:20:00
3,2004-05-01 10:30:00
4,2004-05-01 10:30:00


In [40]:
# ((train_tr["구매일시"].dt.hour > 18) & (train_tr["구매일시"].dt.hour < 21)).mean()

In [41]:
# display(train_tr["구매일시"].max())
# display(train_tr["구매일시"].min())

In [42]:
# train_tr["구매일시"].dt.month == 5

In [43]:
# (train_tr["구매일시"].dt.day == 4).value_counts()

In [44]:
# np.mean((train_tr["구매일시"].dt.month == 5) & (train_tr["구매일시"].dt.day == 4))

In [45]:
# 날짜 기준 ID별 5/5 전날 > groupby 구매비용 + 아동 포함된 구매비용

In [46]:
# # 기본 구매 횟수 계산 > 과적합 원인
# train_tr['hour'] = train_tr['구매일시'].dt.hour
# train_tr['date'] = train_tr['구매일시'].dt.date

# children_day = pd.to_datetime('2004-05-05').date()
# parents_day = pd.to_datetime('2004-05-08').date()

# # 시간대 및 기념일 전후 기준 열 생성
# train_tr['18시~21시'] = ((train_tr['hour'] >= 18) & (train_tr['hour'] <= 21)).astype(int)
# train_tr['어린이날_전날'] = (train_tr['date'] == children_day - pd.Timedelta(days=1)).astype(int)
# train_tr['어린이날_당일'] = (train_tr['date'] == children_day).astype(int)
# train_tr['어버이날_전날'] = (train_tr['date'] == parents_day - pd.Timedelta(days=1)).astype(int)
# train_tr['어버이날_당일'] = (train_tr['date'] == parents_day).astype(int)

# # 각 통계 값 계산을 위한 groupby
# agg_dict = {
#     '구매일시': 'count',
#     '18시~21시': ['sum', 'mean'],
#     '어린이날_전날': ['sum', 'mean'],
#     '어린이날_당일': ['sum', 'mean'],
#     '어버이날_전날': ['sum', 'mean'],
#     '어버이날_당일': ['sum', 'mean']
# }

# # groupby 집계 수행
# tmp = train_tr.groupby('ID').agg(agg_dict).reset_index()

# # 컬럼 이름 정리
# tmp.columns = [
#     'ID', '구매횟수',
#     '18시~21시_구매횟수', '18시~21시_구매비율',
#     '어린이날_전날_구매횟수', '어린이날_전날_구매비율',
#     '어린이날_당일_구매횟수', '어린이날_당일_구매비율',
#     '어버이날_전날_구매횟수', '어버이날_전날_구매비율',
#     '어버이날_당일_구매횟수', '어버이날_당일_구매비율'
# ]

# # 최종 병합
# train_ft = train_ft.merge(tmp, on="ID", how='left')
# test_ft = test_ft.merge(tmp, on="ID", how='left')

In [47]:
# train_ft

In [48]:
# test_ft

In [49]:
# # code_test
# agg_list = [
#         # 컬럼명, 집계 방식
#         ('퇴근후_구매비율', lambda x: np.mean((x.dt.hour >= 18) & (x.dt.hour <= 21))),
#         ('2004_어린이날_전날_구매비율', lambda x : np.mean((x.dt.month == 5) & (x.dt.day == 4))), # 아동관련 구매비율
#         ('2004_어린이날_구매비율', lambda x : np.mean((x.dt.month == 5) & (x.dt.day == 5))),
#         ('2004_어버이날_전날_구매비율', lambda x : np.mean((x.dt.month == 5) & (x.dt.day == 7))),
#         ('2004_어버이날_구매비율', lambda x : np.mean((x.dt.month == 5) & (x.dt.day == 8))),
#     ]
# train_tr.groupby('ID')["구매일시"].agg(agg_list).reset_index()

In [50]:
# code_acting
agg_list = [
        # 컬럼명, 집계 방식
        ('퇴근후_구매비율', lambda x: np.mean((x.dt.hour >= 18) & (x.dt.hour <= 21))),
        ('2004_어린이날_전날_구매비율', lambda x : np.mean((x.dt.month == 5) & (x.dt.day == 4))),
        ('2004_어린이날_구매비율', lambda x : np.mean((x.dt.month == 5) & (x.dt.day == 5))),
        ('2004_어버이날_전날_구매비율', lambda x : np.mean((x.dt.month == 5) & (x.dt.day == 7))),
        ('2004_어버이날_구매비율', lambda x : np.mean((x.dt.month == 5) & (x.dt.day == 8))),
        ('2004_빼빼로데이_구매비율', lambda x : np.mean((x.dt.month == 11) & (x.dt.day == 11))),
        ('2005_발렌타인데이_구매비율', lambda x : np.mean((x.dt.month == 2) & (x.dt.day == 14))),
        ('2005_화이트데이_구매비율', lambda x : np.mean((x.dt.month == 3) & (x.dt.day == 14))),
        ('2005_삼일절', lambda x : np.mean((x.dt.month == 3) & (x.dt.day == 1))),
        ('2005_식목일', lambda x : np.mean((x.dt.month == 4) & (x.dt.day == 5))),
    ]

tmp = train_tr.groupby('ID')["구매일시"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left',on="ID")
train_ft.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,...,퇴근후_구매비율,2004_어린이날_전날_구매비율,2004_어린이날_구매비율,2004_어버이날_전날_구매비율,2004_어버이날_구매비율,2004_빼빼로데이_구매비율,2005_발렌타인데이_구매비율,2005_화이트데이_구매비율,2005_삼일절,2005_식목일
0,train_0,16,23,0.250000,0.050000,0.250000,0.400000,0.300000,3,1.818182,...,0.250000,0.0,0.0,0.05,0.0,0.0,0.000000,0.000000,0.000000,0.000000
1,train_1,41,16,0.023810,0.357143,0.166667,0.357143,0.119048,3,2.000000,...,0.309524,0.0,0.0,0.00,0.0,0.0,0.000000,0.000000,0.071429,0.000000
2,train_2,102,6,0.210526,0.464912,0.140351,0.175439,0.219298,0,2.035714,...,0.149123,0.0,0.0,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000
3,train_3,191,3,0.189573,0.379147,0.180095,0.236967,0.203791,3,2.293478,...,0.222749,0.0,0.0,0.00,0.0,0.0,0.009479,0.023697,0.018957,0.004739
4,train_4,55,11,0.258065,0.112903,0.612903,0.209677,0.064516,4,2.296296,...,0.161290,0.0,0.0,0.00,0.0,0.0,0.016129,0.000000,0.000000,0.000000


In [51]:
tmp = test_tr.groupby('ID')["구매일시"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left',on="ID")
test_ft.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,...,퇴근후_구매비율,2004_어린이날_전날_구매비율,2004_어린이날_구매비율,2004_어버이날_전날_구매비율,2004_어버이날_구매비율,2004_빼빼로데이_구매비율,2005_발렌타인데이_구매비율,2005_화이트데이_구매비율,2005_삼일절,2005_식목일
0,test_0,7,32,0.571429,0.285714,0.285714,0.428571,0.000000,1,1.400000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,test_1,4,54,0.250000,0.000000,0.000000,0.750000,0.250000,0,1.333333,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,test_2,4,8,0.000000,0.000000,0.600000,0.400000,0.000000,0,2.500000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,test_3,28,16,0.031250,0.062500,0.468750,0.343750,0.125000,3,2.000000,...,0.531250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,test_4,19,29,0.476190,0.380952,0.285714,0.238095,0.095238,6,2.333333,...,0.571429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
train_ft.shape, test_ft.shape

((14940, 342), (12225, 342))

## 지점 활용

In [53]:
# 전부 기혼 우세

In [54]:
# train_tr['지점코드'].value_counts()

In [55]:
# train_tr['지점코드'].unique()

In [56]:
# # 지점코드가 'A144000' 또는 'A202000'인 방문 여부 컬럼 추가
# train_tr['A144000방문'] = train_tr['지점코드'] == 'A144000'
# train_tr['A202000방문'] = train_tr['지점코드'] == 'A202000'

# #ID별로 각 지점 방문 비율 계산
# visit_ratio = train_tr.groupby('ID').agg(
#     A144000방문비율=('A144000방문', 'mean'),
#     A202000방문비율=('A202000방문', 'mean')
# ).reset_index()

# # 두 지점 방문 비율 합계 추가
# visit_ratio['A144000_A202000방문비율합'] = visit_ratio['A144000방문비율'] + visit_ratio['A202000방문비율']

# # train_ft = train_ft.rename(columns={'A144000방문비율_x': 'A144000방문비율', 'A202000방문비율_x': 'A202000방문비율','A144000_A202000방문비율합_x': 'A144000_A202000방문비율합'})
# # test_ft = test_ft.rename(columns={'A144000방문비율_x': 'A144000방문비율', 'A202000방문비율_x': 'A202000방문비율','A144000_A202000방문비율합_x': 'A144000_A202000방문비율합'})
# # train_ft와 test_ft에 병합
# train_ft = train_ft.merge(visit_ratio, on='ID', how='left')
# test_ft = test_ft.merge(visit_ratio, on='ID', how='left')

# # 결과 확인
# train_ft.iloc[:, -10:]

In [57]:
# test_ft.iloc[:, -10:]

In [58]:
# #code test
# agg_list = [
#           ('', ),
#     ]
# train_tr.groupby('ID')["지점코드"].agg(agg_list).reset_index()

In [59]:
# # code_acting
# agg_list = [
#           ('', ),
#     ]

# tmp = train_tr.groupby('ID')["지점코드"].agg(agg_list).reset_index()
# train_ft = train_ft.merge(tmp, how='left',on="ID")
# train_ft.head()

In [60]:
# tmp = test_tr.groupby('ID')["지점코드"].agg(agg_list).reset_index()
# test_ft = test_ft.merge(tmp, how='left',on="ID")
# test_ft.head()

In [61]:
train_ft.shape, test_ft.shape

((14940, 342), (12225, 342))

## 브랜드코드 활용

In [62]:
# 기혼특징
#   5100, 5110, 5111, 5112
# 공산품포함 브랜드
#   5100, 5338

In [63]:
train_tr[train_tr['대분류'] == '공산품']['브랜드코드'].unique()

array([5100, 5338])

In [64]:
train_tr['브랜드코드'].unique()

array([5100, 5101, 5105, ..., 6994, 6996, 6524])

In [65]:
train_tr['브랜드코드'].value_counts()

,count
브랜드코드,
5100,119088
5127,8860
5173,6517
5116,5682
5138,5277
...,...
6894,1
6892,1
6506,1


In [66]:
# train_tr.groupby('ID')["브랜드코드"].count()

In [67]:
# 공산품_포함
agg_list = [
    ('공산품_포함관련구매비용', 'sum'),
    ('공산품_포함관련구매건수', 'count'),
    ('공산품_포함관련구매평균', 'mean')
]

# train 데이터에 대해 공산품_포함관련 데이터 집계
fac_product_related = train_tr[train_tr['브랜드코드'].isin(['5100,5338'])]
tmp = fac_product_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 공산품_포함관련 데이터 집계
fac_product_related = test_tr[test_tr['브랜드코드'].isin(['5100,5338'])]
tmp = fac_product_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 공산품_포함관련 구매 비율 계산
train_ft['공산품_포함관련구매비용비율'] = train_ft['공산품_포함관련구매비용'] / train_ft['총구매액']
train_ft['공산품_포함관련구매건수비율'] = train_ft['공산품_포함관련구매건수'] / train_ft['구매건수']

test_ft['공산품_포함관련구매비용비율'] = test_ft['공산품_포함관련구매비용'] / test_ft['총구매액']
test_ft['공산품_포함관련구매건수비율'] = test_ft['공산품_포함관련구매건수'] / test_ft['구매건수']

In [68]:
# # code test
# agg_list = [
#              ('', ),
#              ]
# train_tr.groupby('ID')["브랜드코드"].agg(agg_list).reset_index()

In [69]:
# # code acting
# agg_list = [
#              ('', ),
#              ]

# tmp = train_tr.groupby('ID')["브랜드코드"].agg(agg_list).reset_index()
# train_ft = train_ft.merge(tmp, how='left',on="ID")
# train_ft.head()

In [70]:
# tmp = test_tr.groupby('ID')["브랜드코드"].agg(agg_list).reset_index()
# test_ft = test_ft.merge(tmp, how='left',on="ID")
# test_ft.head()

In [71]:
train_ft.shape, test_ft.shape

((14940, 347), (12225, 347))

## 중분류 활용

In [72]:
# 중분류별 구매횟수 평균mean 표준편차std 왜도skew 첨도kurt

In [73]:
# 기혼_특징
# 가전제품 : 가스기기, 가스렌지, 가전특정, 냉장고_세탁기, 냉장고.세탁기, TV_VTR, 냉난방
# 공산_식품 : 건강식품 건식품, 건어물, 조미료
# 생식품 : 곡물 생선 야채 정육 청과, 용기보증
# 여성_패션 : 가방 란제리 국내부띠끄 로얄부틱2F 명품 모피 모피_피혁 모피.피혁 모피_니트 미시케쥬얼 보석 부띠끄 수입부띠끄 수입종합화장품 엘레강스 엘레강스부틱 엘레강스캐주얼 타운웨어 크리스탈
# 남성_패션 : 트래디셔널 내셔날 남성라이센스
# 스포츠 : 골프(국내), 골프(수입), 골프(LC), 골프(NB), 골프 단품, 골프웨어

In [74]:
# 미혼_특징
# 패션 : 교복행사, 영캐주얼, 영캐쥬얼, 타운, 타운단품, 타운웨어, 화장품, 뉴베이직캐주얼, 베이직캐주얼, 시계, 트렌디캐주얼, 피혁토탈(B2),마담SIZE, 장신구
# 음식 : 일반조리, 즉석조리, 패스트푸드, 일용잡화
# 행사 : 단품_행사(캐릭터), 단품행사2
# 악기 : 영창(문화)
# 취미 : 취미소품

In [75]:
# all_middle_list = train_tr['중분류'].unique().tolist()
# len(all_middle_list)

In [76]:
# # ID별 모든중분류 구매가격
# train_tr.groupby(['ID', '중분류'])['구매가격'].sum().reset_index().head(30)

In [77]:
# # ID별 모든중분류 구매액 > ID별 중분류에 대한 총 구매가격
# train_tr.groupby(['ID'])['구매가격'].sum().reset_index()

In [78]:
# 중분류_기혼우세
elecs = ['가스기기', '가스렌지', '가전특정', '냉장고_세탁기', '냉장고.세탁기', 'TV_VTR', '냉난방']
fac_foods = ['건강식품', '건식품']
uncooked_foods = ['곡물', '생선', '야채', '정육', '청과']
fe_fashions = ['가방', '란제리', '국내부띠끄', '로얄부틱2F', '명품', '모피', '모피_피혁', '모피.피혁,' '모피_니트', '미시케쥬얼', '보석', '부띠끄', '수입부띠끄', '수입종합화장품', '엘레강스', '엘레강스부틱', '엘레강스캐주얼', '타운웨어', '크리스탈']
ma_fashions = ['트래디셔널', '내셔날', '남성라이센스']
sports = ['골프(국내)', '골프(수입)', '골프(LC)', '골프(NB)', '골프단품', '골프웨어']
homewares = ['브랜드침구', '주방용품', '침구']

In [79]:
# # ID별 모든중분류에서 가전 제품에 해당되는 value가 있는지 확인
# train_tr.groupby(['ID', '중분류'])['구매가격'].sum().reset_index().groupby('ID')['중분류'].count()

In [80]:
# # 모든 중분류 구매 가격 총합
# price = []
# for all_middle in all_middle_list:
#   price.append(train_tr[train_tr['중분류'] == all_middle]['구매가격'].sum())

# sum(price)

In [81]:
# # 가전 제품 해당 value 총 금액
# elec_price = []
# for elec in elecs:
#   elec_price.append(train_tr[train_tr['중분류'] == elec ]['구매가격'].sum())

# sum(elec_price)

In [82]:
# # 공산_식품 해당 value 총 금액
# fac_price = []
# for fac_food in fac_foods:
#   fac_price.append(train_tr[train_tr['중분류'] == fac_food ]['구매가격'].sum())

# sum(fac_price)

In [83]:
# # 생식품 해당 value 총 금액
# uncooked_price = []
# for uncooked_food in uncooked_foods:
#   uncooked_price.append(train_tr[train_tr['중분류'] == uncooked_food ]['구매가격'].sum())

# sum(uncooked_price)

In [84]:
# # 여성패션 해당 value 총 금액
# fe_price = []
# for fe_fashion in fe_fashions:
#   fe_price.append(train_tr[train_tr['중분류'] == fe_fashion ]['구매가격'].sum())

# sum(uncooked_price)

In [85]:
# # 남성패션 해당 value 총 금액
# ma_price = []
# for ma_fashion in ma_fashions:
#   ma_price.append(train_tr[train_tr['중분류'] == ma_fashion ]['구매가격'].sum())

# sum(ma_price)

In [86]:
# # 스포츠 해당 value 총 금액
# sport_price = []
# for sport in sports:
#   sport_price.append(train_tr[train_tr['중분류'] == sport ]['구매가격'].sum())

# sum(sport_price)

In [87]:
# # 가정용품 해당 value 총 금액
# homeware_price = []
# for homeware in homewares:
#   homeware_price.append(train_tr[train_tr['중분류'] == homeware ]['구매가격'].sum())

# sum(homeware_price)

In [88]:
# train_tr[train_tr['중분류'] == '용기보증']['대분류'].unique()

In [89]:
# train_tr.groupby('ID')['중분류'].nunique()

In [90]:
# train_tr.groupby('ID')['중분류'].count()

In [91]:
train_tr['중분류'].unique()

array(['차류', '화장잡화', '용기보증', '아동복', '전화기_카세트', '일용잡화', '수입종합화장품', '아동',
       '테이프', '수영복', '상품군미지정', '야채', '진케주얼', '단품', 'TV_VCR', '세탁기_냉장고',
       '트래디셔널', '완구(문화)', '캐릭터캐쥬얼', '주방용품', '문구_팬시', '수입부띠끄', '면류', '정육',
       '골프웨어', '유아용품', '패션시계', '골프(국내)', '진캐쥬얼', '색조화장품', '아웃도어', '양말',
       '영트랜드', '핸드백', '골프(LC)', '곡물', '행사_단품', '과자류', '니트웨어', '란제리', '완구',
       '스타킹', '향수', '수입향수', '레이디숍A', '명품', '문화', '단품_행사(트래디셔널)', '신생아',
       '팬시코너(문화)', '우산장갑', '셔츠', '영캐쥬얼', '행사', '칼라드래디셔널', '생선', '아동잡화',
       '스카프', '하이캐쥬얼', '타운웨어', '영캐주얼', '남성잡화', '캐릭터', '소형취사가전', '유아복',
       '헤어ACC', '로얄부틱2F', '타운단품', '캐리어캐쥬얼', '진캐주얼', '침구', '청과', '일반조리',
       '부띠끄', '모자', '디자이너', '캐릭터캐주얼', '캐릭터슈즈', '디자이너부띠끄', '슈즈', '냉장식품',
       '패션란제리', '수입액세서리', '손수건', '잡화', '스포츠웨어', '캐쥬얼구두', '즉석조리', '여성구두',
       '내의', '골프(수입)', '넥타이', '수입ACC', '잡화(문화)', '욕실용품', 'NB제화', 'L_B침구',
       '국산화장품', '디자이너숍', '썬그라스', '수입의류', '트레디셔널캐주얼', '준보석', '싸롱화',
       '트래디셔널캐쥬얼', '화장품', '수입캐주얼', '수입', '미씨캐릭터', '라이센스', '구두임

# 중분류_카테고리 나누기

In [92]:
train_tr[train_tr['중분류'] == '상품개발지원']['대분류'].unique()

array(['잡화파트', '여성의류파트'], dtype=object)

In [93]:
train_tr[train_tr['대분류'] == '잡화파트']['중분류'].unique()

array(['화장잡화', '수입종합화장품', '스타킹', '헤어ACC', '스카프', '썬그라스', '수입의류', '준보석',
       '국내화장품', '넥타이', '모자', '피혁B행사', '색조화장품', '핸드백', '양말', '우산_장갑',
       '상품개발지원', '패션시계', '수입향수', '수입피혁', '보석', '일반ACC', '수입ACC', '가방',
       '손수건', '머플러', 'GBR  지원', '임대핸드백', '피혁소품', '용기보증'], dtype=object)

In [94]:
# 이벤트(발렌타인 데이+화이트 데이+빼빼로데이)
#   과자류 과자

# 기념일(크리스마스+생일)
#   핸드백 수입종합화장품 색조화장품 향수 수입향수 화장품 국내화장품 행사핸드백 임대핸드백 핸드백임대 레이디숍A 명품 팬시코너(문화) 로얄부틱2F 부띠끄 디자이너부띠끄
#   수입ACC 잡화(문화) 수입의류 준보석 엘레강스부틱 팬시 크리스탈 초도자기 일반악세사리 국내부띠끄 헤어ACC 보석 패션악세사리 헤어악세사리 헤어액세사리 엘레강스
#   임대화장품 수입슈즈 수입피혁 주류 시계 장신구 수입악세사리 수입명품 수입의류행사 패션잡화 토탈부틱 로얄수입행사 수입행사 카메라 GBR  지원 부띠끄행사 로얄

# 절기식품(추석+설+단오+초복+etc...)
#   야채 곡물 정육 생선 가정잡화 식기 과자류 일반식품명품 건어물 주류 용기보증

# 여름
#   수영복 선글라스 선글래스 스카프 냉난방 아웃도어 우산장갑 스포츠웨어 레포츠단품 스포츠캐주얼 스포츠슈즈 우산_장갑 음료 스포츠용품 스포츠단품 레포츠 스포츠용퓸 유니섹스캐주얼
#   홈데코

# 겨울
#   니트웨어 니트 스키 머플러 아웃도어 우산장갑 스포츠웨어 레포츠단품 스포츠캐주얼 스포츠슈즈 우산_장갑 음료 스포츠용품 스포츠단품 레포츠 스포츠용퓸 유니섹스캐주얼 홈데코

# 혼수장만
#   전화기_카세트 TV_VCR 세탁기_냉장고 주방용품 소형취사가전 침구 욕실용품 취사소형 가전특정 수예_인테리어소품 브랜드침구 가정잡화 냉장고.세탁기 전문가구(가구) 홈데코
#   냉장고_세탁기 식기 식탁_소품 소형전기 소형가구(가구) 컴퓨터 전화기 냉난방 침대 수예침장 라디오.카세트 소파 카메라 가스렌지 장식액자 조명 직수입침구 수입_종합가구
#   가스기기 L_B침구 TV_VTR

# 패션 위크(S/S, F/W)
#   진케주얼 수입부띠끄 진캐쥬얼 영캐쥬얼 하이캐쥬얼 타운웨어 영캐주얼 타운단품 진캐주얼 부띠끄 디자이너 디자이너부띠끄 패션란제리 란제리 수입의류 트래디셔널캐쥬얼
#   수입캐주얼 트레디셔널캐주얼 타운 미시케쥬얼 디자이너니트 영커리어캐주얼 국내부띠끄 트렌드캐주얼 유니섹스캐주얼 인텔리젼스 디자이너캐주얼 뉴베이직캐주얼
#   엘레강스캐주얼 DC캐주얼 캐쥬얼단품 TOP디자이너 캐주얼단품

# 새학기(개학+개강)
#   테이프 완구 문구 문구_팬시 팬시코너(문화) 문구(문화) 사무용품 레코드(문화) 교복행사 영캐쥬얼 영캐주얼 영트랜드 하이캐쥬얼 캐릭터캐쥬얼 캐리어캐쥬얼 유니섹스캐주얼
#   디자이너캐주얼 미시케쥬얼 베이직캐주얼 인텔리젼스캐주얼 핸드백 스카프 헤어ACC 헤어악세사리 패션ACC 패션악세사리 일반악세사리 잡화 우산장갑 소형취사가전 소형전기
#   컴퓨터 카메라 가스렌지 아동복 유아용품  아동잡화 교복행사 스포츠웨어 스포츠용품 가방

# 어린이(아동+유아)
#   아동 유아 원목_주니어 원목(주니어) 유아발육기 영잡화 유아복 아동복 아동잡화 신생아 아동특선 아동단품 내의 유아용품 헤어ACC 모자 우산장갑
#   완구(문화) 완구 캐릭터 캐릭터캐주얼 캐릭터슈즈 팬시코너(문화) 문구_팬시 문구(문화) 유아발육기 욕실용품 가정잡화 식기 침구 스포츠캐주얼
#   스포츠용품 교복행사

# 어버이날
#   장신구 패션ACC 패션악세사리 패션시계 핸드백 시계 준보석 크리스탈 선글라스 머플러 스카프 니트웨어 내의 남성구두 여성구두 드레스구두 영캐주얼 트레디셔널캐주얼
#   하이캐쥬얼 엘레강스캐주얼 침구 홈데코 주방용품 욕실용품 소형전기 냉장고_세탁기 가스렌지 식탁_소품 조명 건강식품 건식품 냉장식품 조미료 냉동식품
#   통병조림 문구_팬시

# 바겐세일
#   행사_단품 단품_행사(트래디셔널) 행사 피혁B행사 행사핸드백 단품_행사1 단품_행사2 행사소품 단품_행사 핸드백행사 피혁A행사 수입의류행사 수입행사 행사슈즈 수예행사
#   구두행사 로얄수입행사 피혁소품행사 교복행사 장신구행사업체 정장행사 단품행사 행사구두(5F) 부띠끄행사 특선행사 단품_행사(캐릭터)

# 모임(연말+연초)
#   주류 건어물 냉장식품 인스턴트식품 냉동식품 과자류 음료 즉석조리 조미료 패스트푸드 기타식품 드레스구두 여성구두 남성구두 핸드백 스카프 넥타이 시계 머플러 캐쥬얼구두
#   패션ACC 패션시계 장신구 헤어ACC 모자 스타킹 란제리 TV_VCR 세탁기_냉장고 소형전기 냉장고_세탁기 가스렌지 카세트_전화기 오디오 라디오.카세트 조명 식탁 소파 문구
#   우산장갑 욕실용품 주방용품 침구 홈데코 가정잡화 수예_인테리어소품 잡화 단품 행사_단품 단품행사 특선행사 단품_행사1 단품_행사2 행사업체 행사소품 특판

# 스포츠행사(올림픽+월드컵+아시안게임+실내)
#   스포츠웨어 스포츠캐주얼 스포츠단품 스포츠슈즈 골프웨어 골프(국내) 골프(수입) 영캐쥬얼 트레디셔널캐주얼 영캐주얼 캐릭터캐쥬얼 유니섹스캐주얼 스포츠용품 골프(용품)
#   레포츠 스키 레포츠단품 골프단품 피혁소품행사 스포츠용퓸 모자 우산장갑 선글라스 패션ACC 장신구

# 취업(공채모집+직장)
#   양말 스타킹 셔츠 타운웨어 타운단품 남성잡화 캐쥬얼구두 여성구두 구두임대 타운 임대구두 수입구두 팬시 베이직캐주얼 피혁토탈(B2) 어덜트 영커리어캐주얼 트렌드캐주얼
#   남성구두 가방 수입피혁 피혁A행사 의류기타 구두행사 피혁소품행사 사무용품 피혁소품 정장행사 모피_피혁 모피.피혁 지갑_벨트 행사구두(5F) 구두수선 피혁 정장

# 축제(벚꽃+단풍+불꽃+지역)
#   음료 과자류 인스턴트식품 냉장식품 냉동식품 즉석조리 건어물 주류 드레스구두 캐쥬얼구두 여성구두 남성구두 핸드백 스카프 모자 선글라스 패션ACC 패션시계 장신구 머플러
#   스포츠용품 스포츠캐주얼 레포츠 캐쥬얼단품 유니섹스캐주얼 우산장갑 잡화 욕실용품 홈데코 행사_단품 단품행사 특선행사 행사소품 단품_행사1 단품_행사2 문구_팬시 문구
#   완구(문화) 팬시코너(문화)

# 가전제품
#   냉장고 세탁기 소형전기 가스렌지 주방용품 식탁_소품 에어컨 세탁기_냉장고 소형취사가전 TV_VCR 비디오 컴퓨터 라디오 조명 소형가구(가구)

# 인테리어
#   수예_인테리어소품 홈데코 장식액자 조명 가전특정 가정잡화 소형가구(가구) 수입_종합가구 가구 카페트 소파 식탁 침대

# 중분류_카테고리 피처 생성

In [95]:
# 이벤트
agg_list = [
    ('이벤트관련구매비용', 'sum'),
    ('이벤트관련구매건수', 'count'),
    ('이벤트관련구매평균', 'mean')
]

# train 데이터에 대해 이벤트관련 데이터 집계
events_related = train_tr[train_tr['중분류'].str.contains('과자류|과자', na=False)]
tmp = events_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 이벤트관련 데이터 집계
events_related = test_tr[test_tr['중분류'].str.contains('과자류|과자', na=False)]
tmp = events_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 이벤트관련 구매 비율 계산
train_ft['이벤트관련구매비용비율'] = train_ft['이벤트관련구매비용'] / train_ft['총구매액']
train_ft['이벤트관련구매건수비율'] = train_ft['이벤트관련구매건수'] / train_ft['구매건수']

test_ft['이벤트관련구매비용비율'] = test_ft['이벤트관련구매비용'] / test_ft['총구매액']
test_ft['이벤트관련구매건수비율'] = test_ft['이벤트관련구매건수'] / test_ft['구매건수']

In [96]:
# 기념일(크리스마스+생일+etc...)
agg_list = [
    ('기념일관련구매비용', 'sum'),
    ('기념일관련구매건수', 'count'),
    ('기념일관련구매평균', 'mean')
]

# train 데이터에 대해 기념일관련 데이터 집계
anniversaries_related = train_tr[train_tr['중분류'].str.contains('핸드백|수입종합화장품|색조화장품|향수|수입향수|화장품|국내화장품|행사핸드백|임대핸드백|핸드백임대|레이디숍A|명품|팬시코너(문화)|로얄부틱2F|부띠끄|디자이너부띠끄|수입ACC|잡화(문화)|수입의류|준보석|엘레강스부틱|팬시|크리스탈|초도자기|일반악세사리|국내부띠끄|헤어ACC|보석|패션악세사리|헤어악세사리|헤어액세사리|엘레강스|임대화장품|수입슈즈|수입피혁|주류|시계|장신구|수입악세사리|수입명품|수입의류행사|패션잡화|토탈부틱|로얄수입행사|수입행사|카메라|GBR|지원|부띠끄행사|로얄', na=False)]
tmp = anniversaries_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 기념일관련 데이터 집계
anniversaries_related = test_tr[test_tr['중분류'].str.contains('핸드백|수입종합화장품|색조화장품|향수|수입향수|화장품|국내화장품|행사핸드백|임대핸드백|핸드백임대|레이디숍A|명품|팬시코너(문화)|로얄부틱2F|부띠끄|디자이너부띠끄|수입ACC|잡화(문화)|수입의류|준보석|엘레강스부틱|팬시|크리스탈|초도자기|일반악세사리|국내부띠끄|헤어ACC|보석|패션악세사리|헤어악세사리|헤어액세사리|엘레강스|임대화장품|수입슈즈|수입피혁|주류|시계|장신구|수입악세사리|수입명품|수입의류행사|패션잡화|토탈부틱|로얄수입행사|수입행사|카메라|GBR|지원|부띠끄행사|로얄', na=False)]
tmp = anniversaries_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 기념일관련 구매 비율 계산
train_ft['기념일관련구매비용비율'] = train_ft['기념일관련구매비용'] / train_ft['총구매액']
train_ft['기념일관련구매건수비율'] = train_ft['기념일관련구매건수'] / train_ft['구매건수']

test_ft['기념일관련구매비용비율'] = test_ft['기념일관련구매비용'] / test_ft['총구매액']
test_ft['기념일관련구매건수비율'] = test_ft['기념일관련구매건수'] / test_ft['구매건수']

<ipython-input-96-25cbc8a1a18c>:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  anniversaries_related = train_tr[train_tr['중분류'].str.contains('핸드백|수입종합화장품|색조화장품|향수|수입향수|화장품|국내화장품|행사핸드백|임대핸드백|핸드백임대|레이디숍A|명품|팬시코너(문화)|로얄부틱2F|부띠끄|디자이너부띠끄|수입ACC|잡화(문화)|수입의류|준보석|엘레강스부틱|팬시|크리스탈|초도자기|일반악세사리|국내부띠끄|헤어ACC|보석|패션악세사리|헤어악세사리|헤어액세사리|엘레강스|임대화장품|수입슈즈|수입피혁|주류|시계|장신구|수입악세사리|수입명품|수입의류행사|패션잡화|토탈부틱|로얄수입행사|수입행사|카메라|GBR|지원|부띠끄행사|로얄', na=False)]
<ipython-input-96-25cbc8a1a18c>:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  anniversaries_related = test_tr[test_tr['중분류'].str.contains('핸드백|수입종합화장품|색조화장품|향수|수입향수|화장품|국내화장품|행사핸드백|임대핸드백|핸드백임대|레이디숍A|명품|팬시코너(문화)|로얄부틱2F|부띠끄|디자이너부띠끄|수입ACC|잡화(문화)|수입의류|준보석|엘레강스부틱|팬시|크리스탈|초도자기|일반악세사리|국내부띠끄|헤어ACC|보석|패션악세사리|헤어악세사리|헤어액세사리|엘레강스|임대화장품|수입슈즈|수입피혁|주류|시계|장신구|수입악세사리|수입명품|수입의류행사|패션잡화|토탈부틱|로얄수입행사|수입행사|카메라|GB

In [97]:
# 절기식품(설+추석+단오+초복+etc...)
agg_list = [
    ('절기식품관련구매비용', 'sum'),
    ('절기식품관련구매건수', 'count'),
    ('절기식품관련구매평균', 'mean')
]

# train 데이터에 대해 절기식품관련 데이터 집계
seasonal_related = train_tr[train_tr['중분류'].str.contains('야채|곡물|정육|생선|가정잡화|식기|과자류|일반식품명품|건어물|주류|용기보증', na=False)]
tmp = seasonal_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 절기식품관련 데이터 집계
seasonal_related = test_tr[test_tr['중분류'].str.contains('야채|곡물|정육|생선|가정잡화|식기|과자류|일반식품명품|건어물|주류|용기보증', na=False)]
tmp = seasonal_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 절기식품관련 구매 비율 계산
train_ft['절기식품관련구매비용비율'] = train_ft['절기식품관련구매비용'] / train_ft['총구매액']
train_ft['절기식품관련구매건수비율'] = train_ft['절기식품관련구매건수'] / train_ft['구매건수']

test_ft['절기식품관련구매비용비율'] = test_ft['절기식품관련구매비용'] / test_ft['총구매액']
test_ft['절기식품관련구매건수비율'] = test_ft['절기식품관련구매건수'] / test_ft['구매건수']

In [98]:
# 여름
agg_list = [
    ('여름관련구매비용', 'sum'),
    ('여름관련구매건수', 'count'),
    ('여름관련구매평균', 'mean')
]

# train 데이터에 대해 여름관련 데이터 집계
summer_related = train_tr[train_tr['중분류'].str.contains('수영복|선글라스|선글래스|스카프|냉난방|아웃도어|우산장갑|스포츠웨어|레포츠단품|스포츠캐주얼|스포츠슈즈|우산_장갑|음료|스포츠용품|스포츠단품|레포츠|스포츠용퓸|유니섹스캐주얼|홈데코', na=False)]
tmp = summer_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 여름관련 데이터 집계
summer_related = test_tr[test_tr['중분류'].str.contains('수영복|선글라스|선글래스|스카프|냉난방|아웃도어|우산장갑|스포츠웨어|레포츠단품|스포츠캐주얼|스포츠슈즈|우산_장갑|음료|스포츠용품|스포츠단품|레포츠|스포츠용퓸|유니섹스캐주얼|홈데코', na=False)]
tmp = summer_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 여름관련 구매 비율 계산
train_ft['여름관련구매비용비율'] = train_ft['여름관련구매비용'] / train_ft['총구매액']
train_ft['여름관련구매건수비율'] = train_ft['여름관련구매건수'] / train_ft['구매건수']

test_ft['여름관련구매비용비율'] = test_ft['여름관련구매비용'] / test_ft['총구매액']
test_ft['여름관련구매건수비율'] = test_ft['여름관련구매건수'] / test_ft['구매건수']

In [99]:
# 겨울
agg_list = [
    ('겨울관련구매비용', 'sum'),
    ('겨울관련구매건수', 'count'),
    ('겨울관련구매평균', 'mean')
]

# train 데이터에 대해 겨울관련 데이터 집계
winter_related = train_tr[train_tr['중분류'].str.contains('니트웨어|니트|스키|머플러|아웃도어|우산장갑|스포츠웨어|레포츠단품|스포츠캐주얼|스포츠슈즈|우산_장갑|음료|스포츠용품|스포츠단품|레포츠|스포츠용퓸|유니섹스캐주얼|홈데코', na=False)]
tmp = winter_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 겨울관련 데이터 집계
winter_related = test_tr[test_tr['중분류'].str.contains('과자류|과니트웨어|니트|스키|머플러|아웃도어|우산장갑|스포츠웨어|레포츠단품|스포츠캐주얼|스포츠슈즈|우산_장갑|음료|스포츠용품|스포츠단품|레포츠|스포츠용퓸|유니섹스캐주얼|홈데코자', na=False)]
tmp = winter_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 겨울관련 구매 비율 계산
train_ft['겨울관련구매비용비율'] = train_ft['겨울관련구매비용'] / train_ft['총구매액']
train_ft['겨울관련구매건수비율'] = train_ft['겨울관련구매건수'] / train_ft['구매건수']

test_ft['겨울관련구매비용비율'] = test_ft['겨울관련구매비용'] / test_ft['총구매액']
test_ft['겨울관련구매건수비율'] = test_ft['겨울관련구매건수'] / test_ft['구매건수']

In [100]:
# 혼수장만
agg_list = [
    ('혼수장만관련구매비용', 'sum'),
    ('혼수장만관련구매건수', 'count'),
    ('혼수장만관련구매평균', 'mean')
]

# train 데이터에 대해 혼수장만관련 데이터 집계
wedding_related = train_tr[train_tr['중분류'].str.contains('전화기_카세트|TV_VCR|세탁기_냉장고|주방용품|소형취사가전|침구|욕실용품|취사소형|가전특정|수예_인테리어소품|브랜드침구|가정잡화|냉장고.세탁기|전문가구(가구)|홈데코|냉장고_세탁기|식기|식탁_소품|소형전기|소형가구(가구)|컴퓨터|전화기|냉난방|침대|수예침장|라디오.카세트|소파|카메라|가스렌지|장식액자|조명|직수입침구|수입_종합가구|가스기기|L_B침구|TV_VTR', na=False)]
tmp = wedding_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 혼수장만관련 데이터 집계
wedding_related = test_tr[test_tr['중분류'].str.contains('전화기_카세트|TV_VCR|세탁기_냉장고|주방용품|소형취사가전|침구|욕실용품|취사소형|가전특정|수예_인테리어소품|브랜드침구|가정잡화|냉장고.세탁기|전문가구(가구)|홈데코|냉장고_세탁기|식기|식탁_소품|소형전기|소형가구(가구)|컴퓨터|전화기|냉난방|침대|수예침장|라디오.카세트|소파|카메라|가스렌지|장식액자|조명|직수입침구|수입_종합가구|가스기기|L_B침구|TV_VTR', na=False)]
tmp = wedding_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 혼수장만관련 구매 비율 계산
train_ft['혼수장만관련구매비용비율'] = train_ft['혼수장만관련구매비용'] / train_ft['총구매액']
train_ft['혼수장만관련구매건수비율'] = train_ft['혼수장만관련구매건수'] / train_ft['구매건수']

test_ft['혼수장만관련구매비용비율'] = test_ft['혼수장만관련구매비용'] / test_ft['총구매액']
test_ft['혼수장만관련구매건수비율'] = test_ft['혼수장만관련구매건수'] / test_ft['구매건수']

<ipython-input-100-5f8ccf19f1da>:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  wedding_related = train_tr[train_tr['중분류'].str.contains('전화기_카세트|TV_VCR|세탁기_냉장고|주방용품|소형취사가전|침구|욕실용품|취사소형|가전특정|수예_인테리어소품|브랜드침구|가정잡화|냉장고.세탁기|전문가구(가구)|홈데코|냉장고_세탁기|식기|식탁_소품|소형전기|소형가구(가구)|컴퓨터|전화기|냉난방|침대|수예침장|라디오.카세트|소파|카메라|가스렌지|장식액자|조명|직수입침구|수입_종합가구|가스기기|L_B침구|TV_VTR', na=False)]
<ipython-input-100-5f8ccf19f1da>:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  wedding_related = test_tr[test_tr['중분류'].str.contains('전화기_카세트|TV_VCR|세탁기_냉장고|주방용품|소형취사가전|침구|욕실용품|취사소형|가전특정|수예_인테리어소품|브랜드침구|가정잡화|냉장고.세탁기|전문가구(가구)|홈데코|냉장고_세탁기|식기|식탁_소품|소형전기|소형가구(가구)|컴퓨터|전화기|냉난방|침대|수예침장|라디오.카세트|소파|카메라|가스렌지|장식액자|조명|직수입침구|수입_종합가구|가스기기|L_B침구|TV_VTR', na=False)]


In [101]:
# 패션 위크(S/S, F/W)
agg_list = [
    ('패션위크관련구매비용', 'sum'),
    ('패션위크관련구매건수', 'count'),
    ('패션위크관련구매평균', 'mean')
]

# train 데이터에 대해 패션위크관련 데이터 집계
fashion_related = train_tr[train_tr['중분류'].str.contains('진케주얼|수입부띠끄|진캐쥬얼|영캐쥬얼|하이캐쥬얼|타운웨어|영캐주얼|타운단품|진캐주얼|부띠끄|디자이너|디자이너부띠끄|패션란제리|란제리|수입의류|트래디셔널캐쥬얼|수입캐주얼|트레디셔널캐주얼|타운|미시케쥬얼|디자이너니트|영커리어캐주얼|국내부띠끄|트렌드캐주얼|유니섹스캐주얼|인텔리젼스|디자이너캐주얼|뉴베이직캐주얼|엘레강스캐주얼|DC캐주얼|캐쥬얼단품|TOP디자이너|캐주얼단품', na=False)]
tmp = fashion_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 패션위크관련 데이터 집계
fashion_related = test_tr[test_tr['중분류'].str.contains('진케주얼|수입부띠끄|진캐쥬얼|영캐쥬얼|하이캐쥬얼|타운웨어|영캐주얼|타운단품|진캐주얼|부띠끄|디자이너|디자이너부띠끄|패션란제리|란제리|수입의류|트래디셔널캐쥬얼|수입캐주얼|트레디셔널캐주얼|타운|미시케쥬얼|디자이너니트|영커리어캐주얼|국내부띠끄|트렌드캐주얼|유니섹스캐주얼|인텔리젼스|디자이너캐주얼|뉴베이직캐주얼|엘레강스캐주얼|DC캐주얼|캐쥬얼단품|TOP디자이너|캐주얼단품', na=False)]
tmp = fashion_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 패션위크관련 구매 비율 계산
train_ft['패션위크관련구매비용비율'] = train_ft['패션위크관련구매비용'] / train_ft['총구매액']
train_ft['패션위크관련구매건수비율'] = train_ft['패션위크관련구매건수'] / train_ft['구매건수']

test_ft['패션위크관련구매비용비율'] = test_ft['패션위크관련구매비용'] / test_ft['총구매액']
test_ft['패션위크관련구매건수비율'] = test_ft['패션위크관련구매건수'] / test_ft['구매건수']

In [102]:
# 새학기(개학+개강)
agg_list = [
    ('새학기관련구매비용', 'sum'),
    ('새학기관련구매건수', 'count'),
    ('새학기관련구매평균', 'mean')
]

# train 데이터에 대해 새학기관련 데이터 집계
new_term_related = train_tr[train_tr['중분류'].str.contains('테이프|완구|문구|문구_팬시|팬시코너(문화)|문구(문화)|사무용품|레코드(문화)|교복행사|영캐쥬얼|영캐주얼|영트랜드|하이캐쥬얼|캐릭터캐쥬얼|캐리어캐쥬얼|유니섹스캐주얼|디자이너캐주얼|미시케쥬얼|베이직캐주얼|인텔리젼스캐주얼|핸드백|스카프|헤어ACC|헤어악세사리|패션ACC|패션악세사리|일반악세사리|잡화|우산장갑|소형취사가전|소형전기|컴퓨터|카메라|가스렌지|아동복|유아용품|아동잡화|교복행사|스포츠웨어|스포츠용품|가방', na=False)]
tmp = new_term_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 새학기관련 데이터 집계
new_term_related = test_tr[test_tr['중분류'].str.contains('테이프|완구|문구|문구_팬시|팬시코너(문화)|문구(문화)|사무용품|레코드(문화)|교복행사|영캐쥬얼|영캐주얼|영트랜드|하이캐쥬얼|캐릭터캐쥬얼|캐리어캐쥬얼|유니섹스캐주얼|디자이너캐주얼|미시케쥬얼|베이직캐주얼|인텔리젼스캐주얼|핸드백|스카프|헤어ACC|헤어악세사리|패션ACC|패션악세사리|일반악세사리|잡화|우산장갑|소형취사가전|소형전기|컴퓨터|카메라|가스렌지|아동복|유아용품|아동잡화|교복행사|스포츠웨어|스포츠용품|가방', na=False)]
tmp = new_term_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 새학기관련 구매 비율 계산
train_ft['새학기관련구매비용비율'] = train_ft['새학기관련구매비용'] / train_ft['총구매액']
train_ft['새학기관련구매건수비율'] = train_ft['새학기관련구매건수'] / train_ft['구매건수']

test_ft['새학기관련구매비용비율'] = test_ft['새학기관련구매비용'] / test_ft['총구매액']
test_ft['새학기관련구매건수비율'] = test_ft['새학기관련구매건수'] / test_ft['구매건수']


<ipython-input-102-b53ff4a8f262>:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  new_term_related = train_tr[train_tr['중분류'].str.contains('테이프|완구|문구|문구_팬시|팬시코너(문화)|문구(문화)|사무용품|레코드(문화)|교복행사|영캐쥬얼|영캐주얼|영트랜드|하이캐쥬얼|캐릭터캐쥬얼|캐리어캐쥬얼|유니섹스캐주얼|디자이너캐주얼|미시케쥬얼|베이직캐주얼|인텔리젼스캐주얼|핸드백|스카프|헤어ACC|헤어악세사리|패션ACC|패션악세사리|일반악세사리|잡화|우산장갑|소형취사가전|소형전기|컴퓨터|카메라|가스렌지|아동복|유아용품|아동잡화|교복행사|스포츠웨어|스포츠용품|가방', na=False)]
<ipython-input-102-b53ff4a8f262>:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  new_term_related = test_tr[test_tr['중분류'].str.contains('테이프|완구|문구|문구_팬시|팬시코너(문화)|문구(문화)|사무용품|레코드(문화)|교복행사|영캐쥬얼|영캐주얼|영트랜드|하이캐쥬얼|캐릭터캐쥬얼|캐리어캐쥬얼|유니섹스캐주얼|디자이너캐주얼|미시케쥬얼|베이직캐주얼|인텔리젼스캐주얼|핸드백|스카프|헤어ACC|헤어악세사리|패션ACC|패션악세사리|일반악세사리|잡화|우산장갑|소형취사가전|소형전기|컴퓨터|카메라|가스렌지|아동복|유아용품|아동잡화|교복행사|스포츠웨어|스포츠용품|가방', na=False)]


In [103]:
# 어린이
agg_list = [
    ('어린이관련구매비용', 'sum'),
    ('어린이관련구매건수', 'count'),
    ('어린이관련구매평균', 'mean')
]

# train 데이터에 대해 어린이관련 데이터 집계
children_related = train_tr[train_tr['중분류'].str.contains('아동|유아|원목_주니어|원목(주니어)|유아발육기|영잡화|유아복|아동복|아동잡화|신생아|아동특선|아동단품|내의|유아용품|헤어ACC|모자|우산장갑|완구(문화)|완구|캐릭터|캐릭터캐주얼|캐릭터슈즈|팬시코너(문화)|문구_팬시|문구(문화)|유아발육기|욕실용품|가정잡화|식기|침구|스포츠캐주얼|스포츠용품|교복행사', na=False)]
tmp = children_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 어린이관련 데이터 집계
children_related = test_tr[test_tr['중분류'].str.contains('아동|유아|원목_주니어|원목(주니어)|유아발육기|영잡화|유아복|아동복|아동잡화|신생아|아동특선|아동단품|내의|유아용품|헤어ACC|모자|우산장갑|완구(문화)|완구|캐릭터|캐릭터캐주얼|캐릭터슈즈|팬시코너(문화)|문구_팬시|문구(문화)|유아발육기|욕실용품|가정잡화|식기|침구|스포츠캐주얼|스포츠용품|교복행사', na=False)]
tmp = children_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 어린이관련 구매 비율 계산
train_ft['어린이관련구매비용비율'] = train_ft['어린이관련구매비용'] / train_ft['총구매액']
train_ft['어린이관련구매건수비율'] = train_ft['어린이관련구매건수'] / train_ft['구매건수']

test_ft['어린이관련구매비용비율'] = test_ft['어린이관련구매비용'] / test_ft['총구매액']
test_ft['어린이관련구매건수비율'] = test_ft['어린이관련구매건수'] / test_ft['구매건수']


<ipython-input-103-da060803c942>:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  children_related = train_tr[train_tr['중분류'].str.contains('아동|유아|원목_주니어|원목(주니어)|유아발육기|영잡화|유아복|아동복|아동잡화|신생아|아동특선|아동단품|내의|유아용품|헤어ACC|모자|우산장갑|완구(문화)|완구|캐릭터|캐릭터캐주얼|캐릭터슈즈|팬시코너(문화)|문구_팬시|문구(문화)|유아발육기|욕실용품|가정잡화|식기|침구|스포츠캐주얼|스포츠용품|교복행사', na=False)]
<ipython-input-103-da060803c942>:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  children_related = test_tr[test_tr['중분류'].str.contains('아동|유아|원목_주니어|원목(주니어)|유아발육기|영잡화|유아복|아동복|아동잡화|신생아|아동특선|아동단품|내의|유아용품|헤어ACC|모자|우산장갑|완구(문화)|완구|캐릭터|캐릭터캐주얼|캐릭터슈즈|팬시코너(문화)|문구_팬시|문구(문화)|유아발육기|욕실용품|가정잡화|식기|침구|스포츠캐주얼|스포츠용품|교복행사', na=False)]


In [104]:
# 어버이날
agg_list = [
    ('어버이날관련구매비용', 'sum'),
    ('어버이날관련구매건수', 'count'),
    ('어버이날관련구매평균', 'mean')
]

# train 데이터에 대해 어버이날관련 데이터 집계
parents_day_related = train_tr[train_tr['중분류'].str.contains('장신구|패션ACC|패션악세사리|패션시계|핸드백|시계|준보석|크리스탈|선글라스|머플러|스카프|니트웨어|내의|남성구두|여성구두|드레스구두|영캐주얼|트레디셔널캐주얼|하이캐쥬얼|엘레강스캐주얼|침구|홈데코|주방용품|욕실용품|소형전기|냉장고_세탁기|가스렌지|식탁_소품|조명|건강식품|건식품|냉장식품|조미료|냉동식품|통병조림|문구_팬시', na=False)]
tmp = parents_day_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 어버이날관련 데이터 집계
parents_day_related = test_tr[test_tr['중분류'].str.contains('장신구|패션ACC|패션악세사리|패션시계|핸드백|시계|준보석|크리스탈|선글라스|머플러|스카프|니트웨어|내의|남성구두|여성구두|드레스구두|영캐주얼|트레디셔널캐주얼|하이캐쥬얼|엘레강스캐주얼|침구|홈데코|주방용품|욕실용품|소형전기|냉장고_세탁기|가스렌지|식탁_소품|조명|건강식품|건식품|냉장식품|조미료|냉동식품|통병조림|문구_팬시', na=False)]
tmp = parents_day_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 어버이날관련 구매 비율 계산
train_ft['어버이날관련구매비용비율'] = train_ft['어버이날관련구매비용'] / train_ft['총구매액']
train_ft['어버이날관련구매건수비율'] = train_ft['어버이날관련구매건수'] / train_ft['구매건수']

test_ft['어버이날관련구매비용비율'] = test_ft['어버이날관련구매비용'] / test_ft['총구매액']
test_ft['어버이날관련구매건수비율'] = test_ft['어버이날관련구매건수'] / test_ft['구매건수']


In [105]:
# 바겐세일
agg_list = [
    ('바겐세일관련구매비용', 'sum'),
    ('바겐세일관련구매건수', 'count'),
    ('바겐세일관련구매평균', 'mean')
]

# train 데이터에 대해 바겐세일관련 데이터 집계
bargain_sale_related = train_tr[train_tr['중분류'].str.contains('행사_단품|단품_행사(트래디셔널)|행사|피혁B행사|행사핸드백|단품_행사1|단품_행사2|행사소품|단품_행사|핸드백행사|피혁A행사|수입의류행사|수입행사|행사슈즈|수예행사|구두행사|로얄수입행사|피혁소품행사|교복행사|장신구행사업체|정장행사|단품행사|행사구두(5F)|부띠끄행사|특선행사|단품_행사(캐릭터)', na=False)]
tmp = bargain_sale_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 바겐세일관련 데이터 집계
bargain_sale_related = test_tr[test_tr['중분류'].str.contains('행사_단품|단품_행사(트래디셔널)|행사|피혁B행사|행사핸드백|단품_행사1|단품_행사2|행사소품|단품_행사|핸드백행사|피혁A행사|수입의류행사|수입행사|행사슈즈|수예행사|구두행사|로얄수입행사|피혁소품행사|교복행사|장신구행사업체|정장행사|단품행사|행사구두(5F)|부띠끄행사|특선행사|단품_행사(캐릭터)', na=False)]
tmp = bargain_sale_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 바겐세일관련 구매 비율 계산
train_ft['바겐세일관련구매비용비율'] = train_ft['바겐세일관련구매비용'] / train_ft['총구매액']
train_ft['바겐세일관련구매건수비율'] = train_ft['바겐세일관련구매건수'] / train_ft['구매건수']

test_ft['바겐세일관련구매비용비율'] = test_ft['바겐세일관련구매비용'] / test_ft['총구매액']
test_ft['바겐세일관련구매건수비율'] = test_ft['바겐세일관련구매건수'] / test_ft['구매건수']


<ipython-input-105-23097483acbb>:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  bargain_sale_related = train_tr[train_tr['중분류'].str.contains('행사_단품|단품_행사(트래디셔널)|행사|피혁B행사|행사핸드백|단품_행사1|단품_행사2|행사소품|단품_행사|핸드백행사|피혁A행사|수입의류행사|수입행사|행사슈즈|수예행사|구두행사|로얄수입행사|피혁소품행사|교복행사|장신구행사업체|정장행사|단품행사|행사구두(5F)|부띠끄행사|특선행사|단품_행사(캐릭터)', na=False)]
<ipython-input-105-23097483acbb>:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  bargain_sale_related = test_tr[test_tr['중분류'].str.contains('행사_단품|단품_행사(트래디셔널)|행사|피혁B행사|행사핸드백|단품_행사1|단품_행사2|행사소품|단품_행사|핸드백행사|피혁A행사|수입의류행사|수입행사|행사슈즈|수예행사|구두행사|로얄수입행사|피혁소품행사|교복행사|장신구행사업체|정장행사|단품행사|행사구두(5F)|부띠끄행사|특선행사|단품_행사(캐릭터)', na=False)]


In [106]:
# 모임
agg_list = [
    ('모임관련구매비용', 'sum'),
    ('모임관련구매건수', 'count'),
    ('모임관련구매평균', 'mean')
]

# train 데이터에 대해 모임관련 데이터 집계
meeting_related = train_tr[train_tr['중분류'].str.contains('주류|건어물|냉장식품|인스턴트식품|냉동식품|과자류|음료|즉석조리|조미료|패스트푸드|기타식품|드레스구두|여성구두|남성구두|핸드백|스카프|넥타이|시계|머플러|캐쥬얼구두|패션ACC|패션시계|장신구|헤어ACC|모자|스타킹|란제리|TV_VCR|세탁기_냉장고|소형전기|냉장고_세탁기|가스렌지|카세트_전화기|오디오|라디오.카세트|조명|식탁|소파|문구|우산장갑|욕실용품|주방용품|침구|홈데코|가정잡화|수예_인테리어소품|잡화|단품|행사_단품|단품행사|특선행사|단품_행사1|단품_행사2|행사업체|행사소품|특판', na=False)]
tmp = meeting_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 모임관련 데이터 집계
meeting_related = test_tr[test_tr['중분류'].str.contains('주류|건어물|냉장식품|인스턴트식품|냉동식품|과자류|음료|즉석조리|조미료|패스트푸드|기타식품|드레스구두|여성구두|남성구두|핸드백|스카프|넥타이|시계|머플러|캐쥬얼구두|패션ACC|패션시계|장신구|헤어ACC|모자|스타킹|란제리|TV_VCR|세탁기_냉장고|소형전기|냉장고_세탁기|가스렌지|카세트_전화기|오디오|라디오.카세트|조명|식탁|소파|문구|우산장갑|욕실용품|주방용품|침구|홈데코|가정잡화|수예_인테리어소품|잡화|단품|행사_단품|단품행사|특선행사|단품_행사1|단품_행사2|행사업체|행사소품|특판', na=False)]
tmp = meeting_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 모임관련 구매 비율 계산
train_ft['모임관련구매비용비율'] = train_ft['모임관련구매비용'] / train_ft['총구매액']
train_ft['모임관련구매건수비율'] = train_ft['모임관련구매건수'] / train_ft['구매건수']

test_ft['모임관련구매비용비율'] = test_ft['모임관련구매비용'] / test_ft['총구매액']
test_ft['모임관련구매건수비율'] = test_ft['모임관련구매건수'] / test_ft['구매건수']


In [107]:
# 스포츠행사
agg_list = [
    ('스포츠행사관련구매비용', 'sum'),
    ('스포츠행사관련구매건수', 'count'),
    ('스포츠행사관련구매평균', 'mean')
]

# train 데이터에 대해 스포츠행사관련 데이터 집계
sports_event_related = train_tr[train_tr['중분류'].str.contains('스포츠웨어|스포츠캐주얼|스포츠단품|스포츠슈즈|골프웨어|골프(국내)|골프(수입)|영캐쥬얼|트레디셔널캐주얼|영캐주얼|캐릭터캐주얼|유니섹스캐주얼|스포츠용품|골프(용품)|레포츠|스키|레포츠단품|골프단품|피혁소품행사|스포츠용퓸|모자|우산장갑|선글라스|패션ACC|장신구', na=False)]
tmp = sports_event_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 스포츠행사관련 데이터 집계
sports_event_related = test_tr[test_tr['중분류'].str.contains('스포츠웨어|스포츠캐주얼|스포츠단품|스포츠슈즈|골프웨어|골프(국내)|골프(수입)|영캐쥬얼|트레디셔널캐주얼|영캐주얼|캐릭터캐주얼|유니섹스캐주얼|스포츠용품|골프(용품)|레포츠|스키|레포츠단품|골프단품|피혁소품행사|스포츠용퓸|모자|우산장갑|선글라스|패션ACC|장신구', na=False)]
tmp = sports_event_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 스포츠행사관련 구매 비율 계산
train_ft['스포츠행사관련구매비용비율'] = train_ft['스포츠행사관련구매비용'] / train_ft['총구매액']
train_ft['스포츠행사관련구매건수비율'] = train_ft['스포츠행사관련구매건수'] / train_ft['구매건수']

test_ft['스포츠행사관련구매비용비율'] = test_ft['스포츠행사관련구매비용'] / test_ft['총구매액']
test_ft['스포츠행사관련구매건수비율'] = test_ft['스포츠행사관련구매건수'] / test_ft['구매건수']


<ipython-input-107-b591cf7e7d80>:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  sports_event_related = train_tr[train_tr['중분류'].str.contains('스포츠웨어|스포츠캐주얼|스포츠단품|스포츠슈즈|골프웨어|골프(국내)|골프(수입)|영캐쥬얼|트레디셔널캐주얼|영캐주얼|캐릭터캐주얼|유니섹스캐주얼|스포츠용품|골프(용품)|레포츠|스키|레포츠단품|골프단품|피혁소품행사|스포츠용퓸|모자|우산장갑|선글라스|패션ACC|장신구', na=False)]
<ipython-input-107-b591cf7e7d80>:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  sports_event_related = test_tr[test_tr['중분류'].str.contains('스포츠웨어|스포츠캐주얼|스포츠단품|스포츠슈즈|골프웨어|골프(국내)|골프(수입)|영캐쥬얼|트레디셔널캐주얼|영캐주얼|캐릭터캐주얼|유니섹스캐주얼|스포츠용품|골프(용품)|레포츠|스키|레포츠단품|골프단품|피혁소품행사|스포츠용퓸|모자|우산장갑|선글라스|패션ACC|장신구', na=False)]


In [108]:
# 취업
agg_list = [
    ('취업관련구매비용', 'sum'),
    ('취업관련구매건수', 'count'),
    ('취업관련구매평균', 'mean')
]

# train 데이터에 대해 취업관련 데이터 집계
employment_related = train_tr[train_tr['중분류'].str.contains('양말|스타킹|셔츠|타운웨어|타운단품|남성잡화|캐쥬얼구두|여성구두|구두임대|타운|임대구두|수입구두|팬시|베이직캐주얼|피혁토탈(B2)|어덜트|영커리어캐주얼|트렌드캐주얼|남성구두|가방|수입피혁|피혁A행사|의류기타|구두행사|피혁소품행사|사무용품|피혁소품|정장행사|모피_피혁|모피.피혁|지갑_벨트|행사구두(5F)|구두수선|피혁|정장', na=False)]
tmp = employment_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 취업관련 데이터 집계
employment_related = test_tr[test_tr['중분류'].str.contains('양말|스타킹|셔츠|타운웨어|타운단품|남성잡화|캐쥬얼구두|여성구두|구두임대|타운|임대구두|수입구두|팬시|베이직캐주얼|피혁토탈(B2)|어덜트|영커리어캐주얼|트렌드캐주얼|남성구두|가방|수입피혁|피혁A행사|의류기타|구두행사|피혁소품행사|사무용품|피혁소품|정장행사|모피_피혁|모피.피혁|지갑_벨트|행사구두(5F)|구두수선|피혁|정장', na=False)]
tmp = employment_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 취업관련 구매 비율 계산
train_ft['취업관련구매비용비율'] = train_ft['취업관련구매비용'] / train_ft['총구매액']
train_ft['취업관련구매건수비율'] = train_ft['취업관련구매건수'] / train_ft['구매건수']

test_ft['취업관련구매비용비율'] = test_ft['취업관련구매비용'] / test_ft['총구매액']
test_ft['취업관련구매건수비율'] = test_ft['취업관련구매건수'] / test_ft['구매건수']


<ipython-input-108-e62865dd29b3>:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  employment_related = train_tr[train_tr['중분류'].str.contains('양말|스타킹|셔츠|타운웨어|타운단품|남성잡화|캐쥬얼구두|여성구두|구두임대|타운|임대구두|수입구두|팬시|베이직캐주얼|피혁토탈(B2)|어덜트|영커리어캐주얼|트렌드캐주얼|남성구두|가방|수입피혁|피혁A행사|의류기타|구두행사|피혁소품행사|사무용품|피혁소품|정장행사|모피_피혁|모피.피혁|지갑_벨트|행사구두(5F)|구두수선|피혁|정장', na=False)]
<ipython-input-108-e62865dd29b3>:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  employment_related = test_tr[test_tr['중분류'].str.contains('양말|스타킹|셔츠|타운웨어|타운단품|남성잡화|캐쥬얼구두|여성구두|구두임대|타운|임대구두|수입구두|팬시|베이직캐주얼|피혁토탈(B2)|어덜트|영커리어캐주얼|트렌드캐주얼|남성구두|가방|수입피혁|피혁A행사|의류기타|구두행사|피혁소품행사|사무용품|피혁소품|정장행사|모피_피혁|모피.피혁|지갑_벨트|행사구두(5F)|구두수선|피혁|정장', na=False)]


In [109]:
# 축제
agg_list = [
    ('축제관련구매비용', 'sum'),
    ('축제관련구매건수', 'count'),
    ('축제관련구매평균', 'mean')
]

# train 데이터에 대해 축제관련 데이터 집계
festival_related = train_tr[train_tr['중분류'].str.contains('음료|과자류|인스턴트식품|냉장식품|냉동식품|즉석조리|건어물|주류|드레스구두|캐쥬얼구두|여성구두|남성구두|핸드백|스카프|모자|선글라스|패션ACC|패션시계|장신구|머플러|스포츠용품|스포츠캐주얼|레포츠|캐쥬얼단품|유니섹스캐주얼|우산장갑|잡화|욕실용품|홈데코|행사_단품|단품행사|특선행사|행사소품|단품_행사1|단품_행사2|문구_팬시|문구|완구(문화)|팬시코너(문화)', na=False)]
tmp = festival_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 축제관련 데이터 집계
festival_related = test_tr[test_tr['중분류'].str.contains('음료|과자류|인스턴트식품|냉장식품|냉동식품|즉석조리|건어물|주류|드레스구두|캐쥬얼구두|여성구두|남성구두|핸드백|스카프|모자|선글라스|패션ACC|패션시계|장신구|머플러|스포츠용품|스포츠캐주얼|레포츠|캐쥬얼단품|유니섹스캐주얼|우산장갑|잡화|욕실용품|홈데코|행사_단품|단품행사|특선행사|행사소품|단품_행사1|단품_행사2|문구_팬시|문구|완구(문화)|팬시코너(문화)', na=False)]
tmp = festival_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 축제관련 구매 비율 계산
train_ft['축제관련구매비용비율'] = train_ft['축제관련구매비용'] / train_ft['총구매액']
train_ft['축제관련구매건수비율'] = train_ft['축제관련구매건수'] / train_ft['구매건수']

test_ft['축제관련구매비용비율'] = test_ft['축제관련구매비용'] / test_ft['총구매액']
test_ft['축제관련구매건수비율'] = test_ft['축제관련구매건수'] / test_ft['구매건수']


<ipython-input-109-61075b392966>:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  festival_related = train_tr[train_tr['중분류'].str.contains('음료|과자류|인스턴트식품|냉장식품|냉동식품|즉석조리|건어물|주류|드레스구두|캐쥬얼구두|여성구두|남성구두|핸드백|스카프|모자|선글라스|패션ACC|패션시계|장신구|머플러|스포츠용품|스포츠캐주얼|레포츠|캐쥬얼단품|유니섹스캐주얼|우산장갑|잡화|욕실용품|홈데코|행사_단품|단품행사|특선행사|행사소품|단품_행사1|단품_행사2|문구_팬시|문구|완구(문화)|팬시코너(문화)', na=False)]
<ipython-input-109-61075b392966>:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  festival_related = test_tr[test_tr['중분류'].str.contains('음료|과자류|인스턴트식품|냉장식품|냉동식품|즉석조리|건어물|주류|드레스구두|캐쥬얼구두|여성구두|남성구두|핸드백|스카프|모자|선글라스|패션ACC|패션시계|장신구|머플러|스포츠용품|스포츠캐주얼|레포츠|캐쥬얼단품|유니섹스캐주얼|우산장갑|잡화|욕실용품|홈데코|행사_단품|단품행사|특선행사|행사소품|단품_행사1|단품_행사2|문구_팬시|문구|완구(문화)|팬시코너(문화)', na=False)]


In [110]:
# 가전제품
agg_list = [
    ('가전제품관련구매비용', 'sum'),
    ('가전제품관련구매건수', 'count'),
    ('가전제품관련구매평균', 'mean')
]

# train 데이터에 대해 가전제품관련 데이터 집계
appliance_related = train_tr[train_tr['중분류'].str.contains('냉장고|세탁기|소형전기|가스렌지|주방용품|식탁_소품|에어컨|세탁기_냉장고|소형취사가전|TV_VCR|비디오|컴퓨터|라디오|조명|소형가구(가구)', na=False)]
tmp = appliance_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 가전제품관련 데이터 집계
appliance_related = test_tr[test_tr['중분류'].str.contains('냉장고|세탁기|소형전기|가스렌지|주방용품|식탁_소품|에어컨|세탁기_냉장고|소형취사가전|TV_VCR|비디오|컴퓨터|라디오|조명|소형가구(가구)', na=False)]
tmp = appliance_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 가전제품관련 구매 비율 계산
train_ft['가전제품관련구매비용비율'] = train_ft['가전제품관련구매비용'] / train_ft['총구매액']
train_ft['가전제품관련구매건수비율'] = train_ft['가전제품관련구매건수'] / train_ft['구매건수']

test_ft['가전제품관련구매비용비율'] = test_ft['가전제품관련구매비용'] / test_ft['총구매액']
test_ft['가전제품관련구매건수비율'] = test_ft['가전제품관련구매건수'] / test_ft['구매건수']


<ipython-input-110-81774c2c510c>:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  appliance_related = train_tr[train_tr['중분류'].str.contains('냉장고|세탁기|소형전기|가스렌지|주방용품|식탁_소품|에어컨|세탁기_냉장고|소형취사가전|TV_VCR|비디오|컴퓨터|라디오|조명|소형가구(가구)', na=False)]
<ipython-input-110-81774c2c510c>:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  appliance_related = test_tr[test_tr['중분류'].str.contains('냉장고|세탁기|소형전기|가스렌지|주방용품|식탁_소품|에어컨|세탁기_냉장고|소형취사가전|TV_VCR|비디오|컴퓨터|라디오|조명|소형가구(가구)', na=False)]


In [111]:
# 인테리어
agg_list = [
    ('인테리어관련구매비용', 'sum'),
    ('인테리어관련구매건수', 'count'),
    ('인테리어관련구매평균', 'mean')
]

# train 데이터에 대해 인테리어관련 데이터 집계
interior_related = train_tr[train_tr['중분류'].str.contains('수예_인테리어소품|홈데코|장식액자|조명|가전특정|가정잡화|소형가구(가구)|수입_종합가구|가구|카페트|소파|식탁|침대', na=False)]
tmp = interior_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 인테리어관련 데이터 집계
interior_related = test_tr[test_tr['중분류'].str.contains('수예_인테리어소품|홈데코|장식액자|조명|가전특정|가정잡화|소형가구(가구)|수입_종합가구|가구|카페트|소파|식탁|침대', na=False)]
tmp = interior_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 인테리어관련 구매 비율 계산
train_ft['인테리어관련구매비용비율'] = train_ft['인테리어관련구매비용'] / train_ft['총구매액']
train_ft['인테리어관련구매건수비율'] = train_ft['인테리어관련구매건수'] / train_ft['구매건수']

test_ft['인테리어관련구매비용비율'] = test_ft['인테리어관련구매비용'] / test_ft['총구매액']
test_ft['인테리어관련구매건수비율'] = test_ft['인테리어관련구매건수'] / test_ft['구매건수']


<ipython-input-111-3541f6c6ac82>:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  interior_related = train_tr[train_tr['중분류'].str.contains('수예_인테리어소품|홈데코|장식액자|조명|가전특정|가정잡화|소형가구(가구)|수입_종합가구|가구|카페트|소파|식탁|침대', na=False)]
<ipython-input-111-3541f6c6ac82>:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  interior_related = test_tr[test_tr['중분류'].str.contains('수예_인테리어소품|홈데코|장식액자|조명|가전특정|가정잡화|소형가구(가구)|수입_종합가구|가구|카페트|소파|식탁|침대', na=False)]


In [112]:
train_ft.shape, test_ft.shape

((14940, 432), (12225, 432))

In [113]:
# 각 ID별 총 구매액 계산
total_purchases = train_tr.groupby('ID')['중분류'].count().reset_index(name='total_purchases')

# 주구매_중분류(최빈 중분류)의 구매 횟수 계산
mode_count = train_tr.groupby(['ID', '중분류']).size().reset_index(name='count')
most_frequent = mode_count.loc[mode_count.groupby('ID')['count'].idxmax()].reset_index(drop=True)
most_frequent.rename(columns={'중분류': '주구매_중분류', 'count': '주구매_중분류_구매수'}, inplace=True)

# 총 구매량과 최빈 구매량 병합
purchase_ratio = most_frequent.merge(total_purchases, on='ID')
purchase_ratio['주구매_중분류_구매비율'] = purchase_ratio['주구매_중분류_구매수'] / purchase_ratio['total_purchases']

# 결과를 train_ft, test_ft로 병합
train_ft = train_ft.merge(purchase_ratio[['ID', '주구매_중분류_구매비율']], on='ID', how='left')
test_ft = test_ft.merge(purchase_ratio[['ID', '주구매_중분류_구매비율']], on='ID', how='left')

In [114]:
train_ft.iloc[:,-5:]

,인테리어관련구매건수,인테리어관련구매평균,인테리어관련구매비용비율,인테리어관련구매건수비율,주구매_중분류_구매비율
0,NaN,NaN,NaN,NaN,0.450000
1,NaN,NaN,NaN,NaN,0.142857
2,2.0,83528.0,0.007547,0.017544,0.140351
3,NaN,NaN,NaN,NaN,0.199052
4,NaN,NaN,NaN,NaN,0.193548
...,...,...,...,...,...
14935,NaN,NaN,NaN,NaN,0.368421
14936,NaN,NaN,NaN,NaN,0.250000
14937,NaN,NaN,NaN,NaN,0.125000
14938,NaN,NaN,NaN,NaN,0.393939


In [115]:
test_ft.iloc[:,:-5]

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,...,축제관련구매건수,축제관련구매평균,축제관련구매비용비율,축제관련구매건수비율,가전제품관련구매비용,가전제품관련구매건수,가전제품관련구매평균,가전제품관련구매비용비율,가전제품관련구매건수비율,인테리어관련구매비용
0,test_0,7,32,0.571429,0.285714,0.285714,0.428571,0.000000,1,1.400000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,test_1,4,54,0.250000,0.000000,0.000000,0.750000,0.250000,0,1.333333,...,1.0,118000.00,0.406056,0.250000,NaN,NaN,NaN,NaN,NaN,NaN
2,test_2,4,8,0.000000,0.000000,0.600000,0.400000,0.000000,0,2.500000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,test_3,28,16,0.031250,0.062500,0.468750,0.343750,0.125000,3,2.000000,...,4.0,16700.00,0.028971,0.125000,NaN,NaN,NaN,NaN,NaN,NaN
4,test_4,19,29,0.476190,0.380952,0.285714,0.238095,0.095238,6,2.333333,...,1.0,16200.00,0.010327,0.047619,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12220,test_12220,12,47,0.000000,0.166667,0.166667,0.250000,0.416667,2,2.000000,...,1.0,79000.00,0.049360,0.083333,NaN,NaN,NaN,NaN,NaN,NaN
12221,test_12221,19,24,0.238095,0.380952,0.000000,0.238095,0.380952,0,1.500000,...,1.0,52000.00,0.065648,0.047619,NaN,NaN,NaN,NaN,NaN,NaN
12222,test_12222,38,12,0.315789,0.157895,0.263158,0.289474,0.289474,2,1.357143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12223,test_12223,221,3,0.202335,0.171206,0.311284,0.287938,0.229572,1,2.234783,...,16.0,32131.25,0.032134,0.062257,428440.0,10.0,42844.0,0.026780,0.038911,18900.0


In [116]:
train_ft.shape, test_ft.shape

((14940, 433), (12225, 433))

# 중분류_기혼특징

In [117]:
# elecs = ['가스기기', '가스렌지', '가전특정', '냉장고_세탁기', '냉장고.세탁기', 'TV_VTR', '냉난방']
# fac_foods = ['건강식품', '건식품','조미료']
# uncooked_foods = ['곡물', '생선', '야채', '정육', '청과', '용기보증']
# fe_fashions = ['가방', '란제리', '국내부띠끄', '로얄부틱2F', '명품', '모피', '모피_피혁', '모피.피혁,' '모피_니트', '미시케쥬얼', '보석', '부띠끄', '수입부띠끄', '수입종합화장품', '엘레강스', '엘레강스부틱', '엘레강스캐주얼', '타운웨어', '크리스탈']
# ma_fashions = ['트래디셔널', '내셔날', '남성라이센스']
# sports = ['골프(국내)', '골프(수입)', '골프(LC)', '골프(NB)', '골프단품', '골프웨어']
# middle_homewares = ['브랜드침구', '주방용품', '침구']
# > 구매비율

In [119]:
# 기혼특징_가전제품
agg_list = [
    ('기혼특징_가전제품관련구매비용', 'sum'),
    ('기혼특징_가전제품관련구매건수', 'count'),
    ('기혼특징_가전제품관련구매평균', 'mean')
]

# train 데이터에 대해 기혼특징_가전제품관련 데이터 집계
elecs_related = train_tr[train_tr['중분류'].str.contains('가스기기|가스렌지|가전특정|냉장고_세탁기|명품잡화|냉장고.세탁기|TV_VTR|냉난방', na=False)]
tmp = elecs_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 기혼특징_가전제품관련 데이터 집계
elecs_related = test_tr[test_tr['중분류'].str.contains('가스기기|가스렌지|가전특정|냉장고_세탁기|명품잡화|냉장고.세탁기|TV_VTR|냉난방', na=False)]
tmp = elecs_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 기혼특징_가전제품관련 구매 비율 계산
train_ft['기혼특징_가전제품관련구매비용비율'] = train_ft['기혼특징_가전제품관련구매비용'] / train_ft['총구매액']
train_ft['기혼특징_가전제품관련구매건수비율'] = train_ft['기혼특징_가전제품관련구매건수'] / train_ft['구매건수']

test_ft['기혼특징_가전제품관련구매비용비율'] = test_ft['기혼특징_가전제품관련구매비용'] / test_ft['총구매액']
test_ft['기혼특징_가전제품관련구매건수비율'] = test_ft['기혼특징_가전제품관련구매건수'] / test_ft['구매건수']


In [120]:
# 공산_식품
agg_list = [
    ('공산_식품관련구매비용', 'sum'),
    ('공산_식품관련구매건수', 'count'),
    ('공산_식품관련구매평균', 'mean')
]

# train 데이터에 대해 공산_식품관련 데이터 집계
fac_foods_related = train_tr[train_tr['중분류'].str.contains('건강식품|건식품|건어물|조미료', na=False)]
tmp = fac_foods_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 공산_식품관련 데이터 집계
fac_foods_related = test_tr[test_tr['중분류'].str.contains('건강식품|건식품|건어물|조미료', na=False)]
tmp = fac_foods_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 공산_식품관련 구매 비율 계산
train_ft['공산_식품관련구매비용비율'] = train_ft['공산_식품관련구매비용'] / train_ft['총구매액']
train_ft['공산_식품관련구매건수비율'] = train_ft['공산_식품관련구매건수'] / train_ft['구매건수']

test_ft['공산_식품관련구매비용비율'] = test_ft['공산_식품관련구매비용'] / test_ft['총구매액']
test_ft['공산_식품관련구매건수비율'] = test_ft['공산_식품관련구매건수'] / test_ft['구매건수']

In [121]:
# 생식품
agg_list = [
    ('생식품관련구매비용', 'sum'),
    ('생식품관련구매건수', 'count'),
    ('생식품관련구매평균', 'mean')
]

# train 데이터에 대해 생식품관련 데이터 집계
uncooked_foods_related = train_tr[train_tr['중분류'].str.contains('곡물|생선|야채|정육|청과|용기보증', na=False)]
tmp = uncooked_foods_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 생식품관련 데이터 집계
uncooked_foods_related = test_tr[test_tr['중분류'].str.contains('곡물|생선|야채|정육|청과|용기보증', na=False)]
tmp = uncooked_foods_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 생식품관련 구매 비율 계산
train_ft['생식품관련구매비용비율'] = train_ft['생식품관련구매비용'] / train_ft['총구매액']
train_ft['생식품관련구매건수비율'] = train_ft['생식품관련구매건수'] / train_ft['구매건수']

test_ft['생식품관련구매비용비율'] = test_ft['생식품관련구매비용'] / test_ft['총구매액']
test_ft['생식품관련구매건수비율'] = test_ft['생식품관련구매건수'] / test_ft['구매건수']

In [122]:
# 여성_패션
agg_list = [
    ('여성_패션관련구매비용', 'sum'),
    ('여성_패션관련구매건수', 'count'),
    ('여성_패션관련구매평균', 'mean')
]

# train 데이터에 대해 여성_패션관련 데이터 집계
fe_fashions_related = train_tr[train_tr['중분류'].str.contains('가방|란제리|국내부띠끄|로얄부틱2F|명품|모피|모피_피혁|모피.피혁|모피_니트|미시케쥬얼|보석|부띠끄|수입부띠끄|수입종합화장품|엘레강스|엘레강스부틱|엘레강스캐주얼|타운웨어|크리스탈', na=False)]
tmp = fe_fashions_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 여성_패션관련 데이터 집계
fe_fashions_related = test_tr[test_tr['중분류'].str.contains('가방|란제리|국내부띠끄|로얄부틱2F|명품|모피|모피_피혁|모피.피혁|모피_니트|미시케쥬얼|보석|부띠끄|수입부띠끄|수입종합화장품|엘레강스|엘레강스부틱|엘레강스캐주얼|타운웨어|크리스탈', na=False)]
tmp = fe_fashions_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 여성_패션관련 구매 비율 계산
train_ft['여성_패션관련구매비용비율'] = train_ft['여성_패션관련구매비용'] / train_ft['총구매액']
train_ft['여성_패션관련구매건수비율'] = train_ft['여성_패션관련구매건수'] / train_ft['구매건수']

test_ft['여성_패션관련구매비용비율'] = test_ft['여성_패션관련구매비용'] / test_ft['총구매액']
test_ft['여성_패션관련구매건수비율'] = test_ft['여성_패션관련구매건수'] / test_ft['구매건수']

In [123]:
# 남성_패션
agg_list = [
    ('남성_패션관련구매비용', 'sum'),
    ('남성_패션관련구매건수', 'count'),
    ('남성_패션관련구매평균', 'mean')
]

# train 데이터에 대해 남성_패션관련 데이터 집계
ma_fashions_related = train_tr[train_tr['중분류'].str.contains('트래디셔널|내셔날|남성라이센스', na=False)]
tmp = ma_fashions_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 남성_패션관련 데이터 집계
ma_fashions_related = test_tr[test_tr['중분류'].str.contains('트래디셔널|내셔날|남성라이센스', na=False)]
tmp = ma_fashions_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 남성_패션관련 구매 비율 계산
train_ft['남성_패션관련구매비용비율'] = train_ft['남성_패션관련구매비용'] / train_ft['총구매액']
train_ft['남성_패션관련구매건수비율'] = train_ft['남성_패션관련구매건수'] / train_ft['구매건수']

test_ft['남성_패션관련구매비용비율'] = test_ft['남성_패션관련구매비용'] / test_ft['총구매액']
test_ft['남성_패션관련구매건수비율'] = test_ft['남성_패션관련구매건수'] / test_ft['구매건수']

In [124]:
# 스포츠
agg_list = [
    ('스포츠관련구매비용', 'sum'),
    ('스포츠관련구매건수', 'count'),
    ('스포츠관련구매평균', 'mean')
]

# train 데이터에 대해 스포츠관련 데이터 집계
sports_related = train_tr[train_tr['중분류'].str.contains('골프(국내)|골프(수입)|골프(LC)|골프(NB)|골프단품|골프웨어', na=False)]
tmp = sports_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 스포츠관련 데이터 집계
sports_related = test_tr[test_tr['중분류'].str.contains('골프(국내)|골프(수입)|골프(LC)|골프(NB)|골프단품|골프웨어', na=False)]
tmp = sports_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 스포츠관련 구매 비율 계산
train_ft['스포츠관련구매비용비율'] = train_ft['스포츠관련구매비용'] / train_ft['총구매액']
train_ft['스포츠관련구매건수비율'] = train_ft['스포츠관련구매건수'] / train_ft['구매건수']

test_ft['스포츠관련구매비용비율'] = test_ft['스포츠관련구매비용'] / test_ft['총구매액']
test_ft['스포츠관련구매건수비율'] = test_ft['스포츠관련구매건수'] / test_ft['구매건수']

<ipython-input-124-ab2b1722fb8f>:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  sports_related = train_tr[train_tr['중분류'].str.contains('골프(국내)|골프(수입)|골프(LC)|골프(NB)|골프단품|골프웨어', na=False)]
<ipython-input-124-ab2b1722fb8f>:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  sports_related = test_tr[test_tr['중분류'].str.contains('골프(국내)|골프(수입)|골프(LC)|골프(NB)|골프단품|골프웨어', na=False)]


In [125]:
# 중_가정용품
agg_list = [
    ('중_가정용품관련구매비용', 'sum'),
    ('중_가정용품관련구매건수', 'count'),
    ('중_가정용품관련구매평균', 'mean')
]

# train 데이터에 대해 중_가정용품관련 데이터 집계
middle_homewares_related = train_tr[train_tr['중분류'].str.contains('브랜드침구|주방용품|침구', na=False)]
tmp = middle_homewares_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 중_가정용품관련 데이터 집계
middle_homewares_related = test_tr[test_tr['중분류'].str.contains('브랜드침구|주방용품|침구', na=False)]
tmp = middle_homewares_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 중_가정용품관련 구매 비율 계산
train_ft['중_가정용품관련구매비용비율'] = train_ft['중_가정용품관련구매비용'] / train_ft['총구매액']
train_ft['중_가정용품관련구매건수비율'] = train_ft['중_가정용품관련구매건수'] / train_ft['구매건수']

test_ft['중_가정용품관련구매비용비율'] = test_ft['중_가정용품관련구매비용'] / test_ft['총구매액']
test_ft['중_가정용품관련구매건수비율'] = test_ft['중_가정용품관련구매건수'] / test_ft['구매건수']

In [126]:
train_ft.shape, test_ft.shape

((14940, 471), (12225, 471))

# 중분류_미혼특징

In [127]:
# 미혼_특징
# 패션 : 교복행사, 영캐주얼, 영캐쥬얼, 타운, 타운단품, 타운웨어, 화장품, 뉴베이직캐주얼, 베이직캐주얼, 시계, 트렌디캐주얼, 피혁토탈(B2),마담SIZE, 장신구
# 음식 : 일반조리, 즉석조리, 패스트푸드
# 문화 : 단품행사2 영창(문화) 취미소품

In [128]:
# 패션
agg_list = [
    ('싱글패션관련구매비용', 'sum'),
    ('싱글패션관련구매건수', 'count'),
    ('싱글패션관련구매평균', 'mean')
]

# train 데이터에 대해 싱글패션관련 데이터 집계
single_fashions_related = train_tr[train_tr['중분류'].str.contains('교복행사|영캐주얼|영캐쥬얼|타운|타운단품|타운웨어|화장품|뉴베이직캐주얼|베이직캐주얼|시계|트렌디캐주얼|피혁토탈(B2)|마담SIZE|장신구', na=False)]
tmp = single_fashions_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 싱글패션관련 데이터 집계
single_fashions_related = test_tr[test_tr['중분류'].str.contains('교복행사|영캐주얼|영캐쥬얼|타운|타운단품|타운웨어|화장품|뉴베이직캐주얼|베이직캐주얼|시계|트렌디캐주얼|피혁토탈(B2)|마담SIZE|장신구', na=False)]
tmp = single_fashions_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 싱글패션관련 구매 비율 계산
train_ft['싱글패션관련구매비용비율'] = train_ft['싱글패션관련구매비용'] / train_ft['총구매액']
train_ft['싱글패션관련구매건수비율'] = train_ft['싱글패션관련구매건수'] / train_ft['구매건수']

test_ft['싱글패션관련구매비용비율'] = test_ft['싱글패션관련구매비용'] / test_ft['총구매액']
test_ft['싱글패션관련구매건수비율'] = test_ft['싱글패션관련구매건수'] / test_ft['구매건수']

<ipython-input-128-ded50f5babe8>:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  single_fashions_related = train_tr[train_tr['중분류'].str.contains('교복행사|영캐주얼|영캐쥬얼|타운|타운단품|타운웨어|화장품|뉴베이직캐주얼|베이직캐주얼|시계|트렌디캐주얼|피혁토탈(B2)|마담SIZE|장신구', na=False)]
<ipython-input-128-ded50f5babe8>:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  single_fashions_related = test_tr[test_tr['중분류'].str.contains('교복행사|영캐주얼|영캐쥬얼|타운|타운단품|타운웨어|화장품|뉴베이직캐주얼|베이직캐주얼|시계|트렌디캐주얼|피혁토탈(B2)|마담SIZE|장신구', na=False)]


In [129]:
# 음식
agg_list = [
    ('조리관련구매비용', 'sum'),
    ('조리관련구매건수', 'count'),
    ('조리관련구매평균', 'mean')
]

# train 데이터에 대해 조리관련 데이터 집계
cooked_food_related = train_tr[train_tr['중분류'].str.contains('일반조리|즉석조리|패스트푸드|일용잡화', na=False)]
tmp = cooked_food_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 조리관련 데이터 집계
cooked_food_related = test_tr[test_tr['중분류'].str.contains('일반조리|즉석조리|패스트푸드|일용잡화', na=False)]
tmp = cooked_food_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 조리관련 구매 비율 계산
train_ft['조리관련구매비용비율'] = train_ft['조리관련구매비용'] / train_ft['총구매액']
train_ft['조리관련구매건수비율'] = train_ft['조리관련구매건수'] / train_ft['구매건수']

test_ft['조리관련구매비용비율'] = test_ft['조리관련구매비용'] / test_ft['총구매액']
test_ft['조리관련구매건수비율'] = test_ft['조리관련구매건수'] / test_ft['구매건수']

In [130]:
# 문화
agg_list = [
    ('문화관련구매비용', 'sum'),
    ('문화관련구매건수', 'count'),
    ('문화관련구매평균', 'mean')
]

# train 데이터에 대해 문화관련 데이터 집계
cultures_related = train_tr[train_tr['중분류'].str.contains('단품행사2|영창(문화)|취미소품', na=False)]
tmp = cultures_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 문화관련 데이터 집계
cultures_related = test_tr[test_tr['중분류'].str.contains('단품행사2|영창(문화)|취미소품', na=False)]
tmp = cultures_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 문화관련 구매 비율 계산
train_ft['문화관련구매비용비율'] = train_ft['문화관련구매비용'] / train_ft['총구매액']
train_ft['문화관련구매건수비율'] = train_ft['문화관련구매건수'] / train_ft['구매건수']

test_ft['문화관련구매비용비율'] = test_ft['문화관련구매비용'] / test_ft['총구매액']
test_ft['문화관련구매건수비율'] = test_ft['문화관련구매건수'] / test_ft['구매건수']

<ipython-input-130-ef80547a1d55>:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cultures_related = train_tr[train_tr['중분류'].str.contains('단품행사2|영창(문화)|취미소품', na=False)]
<ipython-input-130-ef80547a1d55>:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cultures_related = test_tr[test_tr['중분류'].str.contains('단품행사2|영창(문화)|취미소품', na=False)]


In [131]:
# code_test
# agg_list = [
            # ('중분류_방문횟수_평균', lambda x: x.count() / x.nunique()),
            # ('가전기기_구매비율', train_tr.apply(lambda x: x['구매가격'] / train_tr['구매가격'].sum() if x['중분류'] == '가스기기' else 0, axis=1))
            # ('가전기기_구매비율', lambda x: train_tr[x == '가스기기']['구매가격'].sum() / train_tr['구매가격'].sum()),
            # ('공산_식품', lambda x: np.mean(x.str.endswith('식품').astype(int))),
            # ('생식품', lambda x: np.mean(x.isin(['곡물', '생선', '야채', '정육', '청과']).astype(int))),
            # ('여성_기혼_패션', lambda x: np.mean(x.isin(fe_cols).astype(int))),
            # ('남성_기혼_패션', lambda x: np.mean(x.isin(ma_cols).astype(int)))
            # ]

# train_tr.groupby('ID')["중분류"].agg(agg_list).reset_index().head(60)

In [132]:
# # code_acting > 수정 및 보완 필요 부분
# agg_list = [
#             ('중분류_방문횟수_비율', lambda x: x.count() / x.nunique()),
#             ('가전제품', lambda x: np.mean(x.str.startswith('가').astype(int))),
#             ('공산_식품', lambda x: np.mean(x.str.endswith('식품').astype(int))),
#             ('생식품', lambda x: np.mean(x.isin(['곡물', '생선', '야채', '정육', '청과']).astype(int))),
#             ('여성_기혼_패션', lambda x: np.mean(x.isin(fe_cols).astype(int))),
#             ('남성_기혼_패션', lambda x: np.mean(x.isin(ma_cols).astype(int)))
#             ]

# tmp = train_tr.groupby('ID')["중분류"].agg(agg_list).reset_index()
# train_ft = train_ft.merge(tmp, how='left')
# train_ft.head()

In [133]:
# tmp = test_tr.groupby('ID')["중분류"].agg(agg_list).reset_index()
# test_ft = test_ft.merge(tmp, how='left')
# test_ft.head()

In [134]:
train_ft.shape, test_ft.shape

((14940, 486), (12225, 486))

##대분류 활용

In [135]:
# 기혼특징_대분류
#   가정 : 가정용품, 가정용품파트, 로얄부띠끄, 로얄부틱
#   골프 : 골프_유니캐주얼
#   명품 : 명품잡화, 로얄부띠끄, 로얄부틱
#   식품 : 생식품, 생식품파트
#   아동 : 아동, 아동_스포츠, 아동문화,
#   잡화 : 잡화, 잡화파트, 패션잡화

# 미혼특징_대분류
#   대_패션: 스포츠_캐주얼, 여성캐주얼, 여성캐쥬얼, 영라이브, 영어덜트캐쥬얼, 영캐릭터, 영플라자

In [136]:
# train_tr['대분류'].unique()

In [137]:
# train_tr['대분류'].isin(['가정용품','골프_유니캐주얼','생식품'])

In [138]:
# cols = ['가정용품', '골프_유니캐주얼', '로얄부띠끄', '로얄부틱', '명품잡화', '생식품']

In [139]:
# train_tr['대분류'].isin(cols).astype(int)

In [140]:
# (train_tr['대분류'].str.startswith('아','영'))

In [141]:
# train_tr['대분류'].str.endswith('아동').value_counts()

In [142]:
# ((train_tr['대분류'].str.startswith('아','영')) & (train_tr['대분류'].str.endswith('아동'))).astype(int)

In [143]:
# # code_test
# agg_list = [
#             ('아동_대분류', lambda x : np.mean(((x.str.startswith('아')) & (x.str.endswith('아동'))).astype(int))),
#             ('여성_기혼', lambda x: np.mean((x.str.startswith('명품','로얄')).astype(int))),
#             ('남녀_기혼', lambda x: np.mean(x.isin(['가정용품','골프_유니캐주얼','생식품']))),


#             ]
# train_tr.groupby('ID')["대분류"].agg(agg_list).reset_index()

In [144]:
# 각 ID별 총 구매액 계산
total_purchases = train_tr.groupby('ID')['대분류'].count().reset_index(name='total_purchases')

# 주구매_대분류(=최빈 대분류)의 구매 횟수 계산
mode_count = train_tr.groupby(['ID', '대분류']).size().reset_index(name='count')
most_frequent = mode_count.loc[mode_count.groupby('ID')['count'].idxmax()].reset_index(drop=True)
most_frequent.rename(columns={'대분류': '주구매_대분류', 'count': '주구매_대분류_구매수'}, inplace=True)

# 총 구매량과 최빈 구매량 병합
purchase_ratio = most_frequent.merge(total_purchases, on='ID')
purchase_ratio['주구매_대분류_구매비율'] = purchase_ratio['주구매_대분류_구매수'] / purchase_ratio['total_purchases']

# 결과를 train_ft, test_ft로 병합
train_ft = train_ft.merge(purchase_ratio[['ID', '주구매_대분류_구매비율']], on='ID', how='left')
test_ft = test_ft.merge(purchase_ratio[['ID', '주구매_대분류_구매비율']], on='ID', how='left')

In [145]:
train_ft.iloc[:,:-5]

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,...,싱글패션관련구매건수,싱글패션관련구매평균,싱글패션관련구매비용비율,싱글패션관련구매건수비율,조리관련구매비용,조리관련구매건수,조리관련구매평균,조리관련구매비용비율,조리관련구매건수비율,문화관련구매비용
0,train_0,16,23,0.250000,0.050000,0.250000,0.400000,0.300000,3,1.818182,...,1.0,169000.000000,0.040227,0.050000,NaN,NaN,NaN,NaN,NaN,NaN
1,train_1,41,16,0.023810,0.357143,0.166667,0.357143,0.119048,3,2.000000,...,14.0,129642.857143,0.359903,0.333333,NaN,NaN,NaN,NaN,NaN,NaN
2,train_2,102,6,0.210526,0.464912,0.140351,0.175439,0.219298,0,2.035714,...,15.0,210866.666667,0.142893,0.131579,38500.0,2.0,19250.0,0.001739,0.017544,NaN
3,train_3,191,3,0.189573,0.379147,0.180095,0.236967,0.203791,3,2.293478,...,50.0,89976.000000,0.126389,0.236967,NaN,NaN,NaN,NaN,NaN,NaN
4,train_4,55,11,0.258065,0.112903,0.612903,0.209677,0.064516,4,2.296296,...,5.0,42200.000000,0.052897,0.080645,66330.0,5.0,13266.0,0.016629,0.080645,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14935,train_14935,16,24,0.210526,0.000000,0.578947,0.052632,0.368421,2,1.900000,...,7.0,116285.714286,0.681103,0.368421,NaN,NaN,NaN,NaN,NaN,NaN
14936,train_14936,8,29,0.125000,0.125000,0.625000,0.250000,0.000000,0,1.000000,...,1.0,52000.000000,0.023423,0.125000,NaN,NaN,NaN,NaN,NaN,NaN
14937,train_14937,22,26,0.083333,0.208333,0.458333,0.208333,0.125000,1,2.000000,...,4.0,72175.000000,0.321529,0.166667,NaN,NaN,NaN,NaN,NaN,NaN
14938,train_14938,31,28,0.151515,0.393939,0.393939,0.212121,0.000000,3,2.750000,...,18.0,167627.777778,0.729415,0.545455,NaN,NaN,NaN,NaN,NaN,NaN


In [146]:
test_ft.iloc[:,:-5]

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,...,싱글패션관련구매건수,싱글패션관련구매평균,싱글패션관련구매비용비율,싱글패션관련구매건수비율,조리관련구매비용,조리관련구매건수,조리관련구매평균,조리관련구매비용비율,조리관련구매건수비율,문화관련구매비용
0,test_0,7,32,0.571429,0.285714,0.285714,0.428571,0.000000,1,1.400000,...,1.0,110000.000000,0.256065,0.142857,NaN,NaN,NaN,NaN,NaN,NaN
1,test_1,4,54,0.250000,0.000000,0.000000,0.750000,0.250000,0,1.333333,...,1.0,73000.000000,0.251204,0.250000,NaN,NaN,NaN,NaN,NaN,NaN
2,test_2,4,8,0.000000,0.000000,0.600000,0.400000,0.000000,0,2.500000,...,2.0,45000.000000,0.148270,0.400000,NaN,NaN,NaN,NaN,NaN,NaN
3,test_3,28,16,0.031250,0.062500,0.468750,0.343750,0.125000,3,2.000000,...,14.0,98542.857143,0.598333,0.437500,NaN,NaN,NaN,NaN,NaN,NaN
4,test_4,19,29,0.476190,0.380952,0.285714,0.238095,0.095238,6,2.333333,...,3.0,138700.000000,0.265253,0.142857,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12220,test_12220,12,47,0.000000,0.166667,0.166667,0.250000,0.416667,2,2.000000,...,8.0,151437.500000,0.756951,0.666667,NaN,NaN,NaN,NaN,NaN,NaN
12221,test_12221,19,24,0.238095,0.380952,0.000000,0.238095,0.380952,0,1.500000,...,2.0,38500.000000,0.097210,0.095238,NaN,NaN,NaN,NaN,NaN,NaN
12222,test_12222,38,12,0.315789,0.157895,0.263158,0.289474,0.289474,2,1.357143,...,2.0,167500.000000,0.023036,0.052632,60000.0,1.0,60000.0,0.004126,0.026316,NaN
12223,test_12223,221,3,0.202335,0.171206,0.311284,0.287938,0.229572,1,2.234783,...,34.0,65523.529412,0.139251,0.132296,24900.0,2.0,12450.0,0.001556,0.007782,NaN


In [147]:
train_ft.shape, test_ft.shape

((14940, 487), (12225, 487))

# 대분류_기혼특징

In [148]:
# 대_가정용품
agg_list = [
    ('대_가정용품관련구매비용', 'sum'),
    ('대_가정용품관련구매건수', 'count'),
    ('대_가정용품관련구매평균', 'mean')
]

# train 데이터에 대해 대_가정용품관련 데이터 집계
major_homewares_related = train_tr[train_tr['대분류'].str.contains('가정용품|가정용품파트(문화)', na=False)]
tmp = major_homewares_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 대_가정용품관련 데이터 집계
major_homewares_related = test_tr[test_tr['대분류'].str.contains('가정용품|가정용품파트(문화)', na=False)]
tmp = major_homewares_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 대_가정용품관련 구매 비율 계산
train_ft['대_가정용품관련구매비용비율'] = train_ft['대_가정용품관련구매비용'] / train_ft['총구매액']
train_ft['대_가정용품관련구매건수비율'] = train_ft['대_가정용품관련구매건수'] / train_ft['구매건수']

test_ft['대_가정용품관련구매비용비율'] = test_ft['대_가정용품관련구매비용'] / test_ft['총구매액']
test_ft['대_가정용품관련구매건수비율'] = test_ft['대_가정용품관련구매건수'] / test_ft['구매건수']

<ipython-input-148-e9b1e54d4ffc>:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  major_homewares_related = train_tr[train_tr['대분류'].str.contains('가정용품|가정용품파트(문화)', na=False)]
<ipython-input-148-e9b1e54d4ffc>:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  major_homewares_related = test_tr[test_tr['대분류'].str.contains('가정용품|가정용품파트(문화)', na=False)]


In [149]:
# 골프_유니
agg_list = [
    ('골프_유니관련구매비용', 'sum'),
    ('골프_유니관련구매건수', 'count'),
    ('골프_유니관련구매평균', 'mean')
]

# train 데이터에 대해 골프_유니관련 데이터 집계
golf_uni_related = train_tr[train_tr['대분류'].str.contains('골프_유니캐주얼', na=False)]
tmp = golf_uni_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 골프_유니관련 데이터 집계
golf_uni_related = test_tr[test_tr['대분류'].str.contains('골프_유니캐주얼', na=False)]
tmp = golf_uni_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 골프_유니관련 구매 비율 계산
train_ft['골프_유니관련구매비용비율'] = train_ft['골프_유니관련구매비용'] / train_ft['총구매액']
train_ft['골프_유니관련구매건수비율'] = train_ft['골프_유니관련구매건수'] / train_ft['구매건수']

test_ft['골프_유니관련구매비용비율'] = test_ft['골프_유니관련구매비용'] / test_ft['총구매액']
test_ft['골프_유니관련구매건수비율'] = test_ft['골프_유니관련구매건수'] / test_ft['구매건수']

In [150]:
# 명품
agg_list = [
    ('명품관련구매비용', 'sum'),
    ('명품관련구매건수', 'count'),
    ('명품관련구매평균', 'mean')
]

# train 데이터에 대해 명품관련 데이터 집계
luxury_related = train_tr[train_tr['대분류'].str.contains('명품잡화|로얄부띠끄|로얄부틱', na=False)]
tmp = luxury_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 명품관련 데이터 집계
luxury_related = test_tr[test_tr['대분류'].str.contains('명품잡화|로얄부띠끄|로얄부틱', na=False)]
tmp = luxury_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 명품관련 구매 비율 계산
train_ft['명품관련구매비용비율'] = train_ft['명품관련구매비용'] / train_ft['총구매액']
train_ft['명품관련구매건수비율'] = train_ft['명품관련구매건수'] / train_ft['구매건수']

test_ft['명품관련구매비용비율'] = test_ft['명품관련구매비용'] / test_ft['총구매액']
test_ft['명품관련구매건수비율'] = test_ft['명품관련구매건수'] / test_ft['구매건수']

In [151]:
# 대_식품
agg_list = [
    ('대_식품관련구매비용', 'sum'),
    ('대_식품관련구매건수', 'count'),
    ('대_식품관련구매평균', 'mean')
]

# train 데이터에 대해 대_식품관련 데이터 집계
major_food_related = train_tr[train_tr['대분류'].str.contains('생식품|생식품파트', na=False)]
tmp = major_food_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 대_식품관련 데이터 집계
major_food_related = test_tr[test_tr['대분류'].str.contains('생식품|생식품파트', na=False)]
tmp = major_food_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 대_식품관련 구매 비율 계산
train_ft['대_식품관련구매비용비율'] = train_ft['대_식품관련구매비용'] / train_ft['총구매액']
train_ft['대_식품관련구매건수비율'] = train_ft['대_식품관련구매건수'] / train_ft['구매건수']

test_ft['대_식품관련구매비용비율'] = test_ft['대_식품관련구매비용'] / test_ft['총구매액']
test_ft['대_식품관련구매건수비율'] = test_ft['대_식품관련구매건수'] / test_ft['구매건수']

# 평균 대_식품 구매액 사분위수 계산
train_ft['평균_대_식품_구매액'] = train_ft['대_식품관련구매비용'] / train_ft['대_식품관련구매건수']
train_ft['평균_대_식품_구매액_Q1'] = train_ft['평균_대_식품_구매액'].quantile(0.25)
train_ft['평균_대_식품_구매액_Q3'] = train_ft['평균_대_식품_구매액'].quantile(0.75)

test_ft['평균_대_식품_구매액'] = test_ft['대_식품관련구매비용'] / test_ft['대_식품관련구매건수']
test_ft['평균_대_식품_구매액_Q1'] = test_ft['평균_대_식품_구매액'].quantile(0.25)
test_ft['평균_대_식품_구매액_Q3'] = test_ft['평균_대_식품_구매액'].quantile(0.75)

In [152]:
# 아동
agg_list = [
    ('아동관련구매비용', 'sum'),
    ('아동관련구매건수', 'count'),
    ('아동관련구매평균', 'mean')
]

# train 데이터에 대해 아동관련 데이터 집계
child_related = train_tr[train_tr['대분류'].str.contains('아동|아동_스포츠|아동문화', na=False)]
tmp = child_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 아동관련 데이터 집계
child_related = test_tr[test_tr['대분류'].str.contains('아동|아동_스포츠|아동문화', na=False)]
tmp = child_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 아동관련 구매 비율 계산
train_ft['아동관련구매비용비율'] = train_ft['아동관련구매비용'] / train_ft['총구매액']
train_ft['아동관련구매건수비율'] = train_ft['아동관련구매건수'] / train_ft['구매건수']

test_ft['아동관련구매비용비율'] = test_ft['아동관련구매비용'] / test_ft['총구매액']
test_ft['아동관련구매건수비율'] = test_ft['아동관련구매건수'] / test_ft['구매건수']

In [153]:
# 잡화
agg_list = [
    ('잡화관련구매비용', 'sum'),
    ('잡화관련구매건수', 'count'),
    ('잡화관련구매평균', 'mean')
]

# train 데이터에 대해 잡화관련 데이터 집계
sundry_related = train_tr[train_tr['대분류'].str.contains('잡화|잡화파트|패션잡화', na=False)]
tmp = sundry_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 잡화관련 데이터 집계
sundry_related = test_tr[test_tr['대분류'].str.contains('잡화|잡화파트|패션잡화', na=False)]
tmp = sundry_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 잡화관련 구매 비율 계산
train_ft['잡화관련구매비용비율'] = train_ft['잡화관련구매비용'] / train_ft['총구매액']
train_ft['잡화관련구매건수비율'] = train_ft['잡화관련구매건수'] / train_ft['구매건수']

test_ft['잡화관련구매비용비율'] = test_ft['잡화관련구매비용'] / test_ft['총구매액']
test_ft['잡화관련구매건수비율'] = test_ft['잡화관련구매건수'] / test_ft['구매건수']

In [154]:
train_ft.shape, test_ft.shape

((14940, 520), (12225, 520))

# 대분류_미혼특징

In [155]:
# # 대_패션
# agg_list = [
#     ('대_패션관련구매비용', 'sum'),
#     ('대_패션관련구매건수', 'count'),
#     ('대_패션관련구매평균', 'mean')
# ]

# # train 데이터에 대해 대_패션관련 데이터 집계
# major_fashion_related = train_tr[train_tr['대분류'].str.contains('스포츠_캐주얼|여성캐주얼|여성캐쥬얼|영라이브|영어덜트캐쥬얼|영캐릭터|영플라자', na=False)]
# tmp = major_fashion_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
# train_ft = train_ft.merge(tmp, how='left', on="ID")

# # test 데이터에 대해 대_패션관련 데이터 집계
# major_fashion_related = test_tr[test_tr['대분류'].str.contains('스포츠_캐주얼|여성캐주얼|여성캐쥬얼|영라이브|영어덜트캐쥬얼|영캐릭터|영플라자', na=False)]
# tmp = major_fashion_related.groupby('ID')['구매가격'].agg(agg_list).reset_index()
# test_ft = test_ft.merge(tmp, how='left', on="ID")

# # 대_패션관련 구매 비율 계산
# train_ft['대_패션관련구매비용비율'] = train_ft['대_패션관련구매비용'] / train_ft['총구매액']
# train_ft['대_패션관련구매건수비율'] = train_ft['대_패션관련구매건수'] / train_ft['구매건수']

# test_ft['대_패션관련구매비용비율'] = test_ft['대_패션관련구매비용'] / test_ft['총구매액']
# test_ft['대_패션관련구매건수비율'] = test_ft['대_패션관련구매건수'] / test_ft['구매건수']

In [156]:
# # code acting
# agg_list = [
#             ('아동_대분류', lambda x : np.mean(((x.str.startswith('아')) & (x.str.endswith('아동'))).astype(int))),
#             ('여성_기혼', lambda x: np.mean((x.str.startswith('명품','로얄')).astype(int))),
#             ('남녀_기혼', lambda x: np.mean(x.isin(['가정용품','골프_유니캐주얼','생식품']))),
#             ('영_아동_대분류', lambda x : np.mean(((x.str.startswith('아')) & (x.str.endswith('아동'))).astype(int)))
#             ]

# tmp = train_tr.groupby('ID')["대분류"].agg(agg_list).reset_index()
# train_ft = train_ft.merge(tmp, how='left')
# train_ft.head()

In [157]:
# tmp = test_tr.groupby('ID')["대분류"].agg(agg_list).reset_index()
# test_ft = test_ft.merge(tmp, how='left')
# test_ft.head()

In [158]:
train_ft.shape, test_ft.shape

((14940, 520), (12225, 520))

## 구매가격 활용

In [159]:
# 구매가격 사분위수, 전체 고객 VIP 10%
# 명품+가전제품의 구매가격 상위 10%, 패션+화장품의 구매가격 상위 10%

In [160]:
train_tr.isnull().sum()

,0
ID,0
구매일시,0
지점코드,0
대분류,0
중분류,0
브랜드코드,0
구매가격,0


In [161]:
# 사분위수 계산 (0.25, 0.75)
agg_list = [
    ('구매가격_사분위수_Q1', lambda x: x.quantile(0.25)),
    ('구매가격_사분위수_Q3', lambda x: x.quantile(0.75))
]

# train 데이터에 대해 사분위수관련 데이터 집계
tmp = train_tr.groupby('ID')['구매가격'].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")

# test 데이터에 대해 사분위수관련 데이터 집계
tmp = test_tr.groupby('ID')['구매가격'].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left', on="ID")

# 각 ID별로 전체 구매가격의 상위 10% 계산
train_top10_threshold = train_tr.groupby('ID')['구매가격'].apply(lambda x: x.nlargest(int(len(x) * 0.1)).min()).reset_index()
train_top10_threshold.columns = ['ID', '상위10%_구매가격']  # 컬럼명 변경
train_ft = train_ft.merge(train_top10_threshold, how='left', on='ID')

test_top10_threshold = test_tr.groupby('ID')['구매가격'].apply(lambda x: x.nlargest(int(len(x) * 0.1)).min()).reset_index()
test_top10_threshold.columns = ['ID', '상위10%_구매가격']  # 컬럼명 변경
test_ft = test_ft.merge(test_top10_threshold, how='left', on='ID')

# 각 ID별 구매가격의 합계 계산
train_sum = train_tr.groupby('ID')['구매가격'].sum().reset_index()
train_sum.columns = ['ID', '구매가격_합계']
train_ft = train_ft.merge(train_sum, how='left', on='ID')

test_sum = test_tr.groupby('ID')['구매가격'].sum().reset_index()
test_sum.columns = ['ID', '구매가격_합계']
test_ft = test_ft.merge(test_sum, how='left', on='ID')

# 상위 10% 여부 판별
train_ft['상위10%_여부'] = (train_ft['구매가격_합계'] >= train_ft['상위10%_구매가격']).astype(int)
test_ft['상위10%_여부'] = (test_ft['구매가격_합계'] >= test_ft['상위10%_구매가격']).astype(int)

In [162]:
# train_ft['상위10%_구매가격'].isnull().sum()

In [163]:
train_ft.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,...,잡화관련구매비용,잡화관련구매건수,잡화관련구매평균,잡화관련구매비용비율,잡화관련구매건수비율,구매가격_사분위수_Q1,구매가격_사분위수_Q3,상위10%_구매가격,구매가격_합계,상위10%_여부
0,train_0,16,23,0.250000,0.050000,0.250000,0.400000,0.300000,3,1.818182,...,697500.0,4.0,174375.000000,0.166024,0.200000,41375.0,362500.0,1076000.0,4201200,1
1,train_1,41,16,0.023810,0.357143,0.166667,0.357143,0.119048,3,2.000000,...,1097000.0,10.0,109700.000000,0.217528,0.238095,37500.0,146000.0,300000.0,5043025,1
2,train_2,102,6,0.210526,0.464912,0.140351,0.175439,0.219298,0,2.035714,...,11956700.0,46.0,259928.260870,0.540162,0.403509,37514.0,189750.0,396000.0,22135386,1
3,train_3,191,3,0.189573,0.379147,0.180095,0.236967,0.203791,3,2.293478,...,7175000.0,65.0,110384.615385,0.201575,0.308057,49000.0,200500.0,456000.0,35594762,1
4,train_4,55,11,0.258065,0.112903,0.612903,0.209677,0.064516,4,2.296296,...,1695000.0,21.0,80714.285714,0.424933,0.338710,12211.0,223750.0,695000.0,3988866,1


In [164]:
test_ft.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,...,잡화관련구매비용,잡화관련구매건수,잡화관련구매평균,잡화관련구매비용비율,잡화관련구매건수비율,구매가격_사분위수_Q1,구매가격_사분위수_Q3,상위10%_구매가격,구매가격_합계,상위10%_여부
0,test_0,7,32,0.571429,0.285714,0.285714,0.428571,0.000000,1,1.400000,...,110000.0,1.0,110000.000000,0.256065,0.142857,37200.0,80000.0,NaN,429579,0
1,test_1,4,54,0.250000,0.000000,0.000000,0.750000,0.250000,0,1.333333,...,110600.0,2.0,55300.000000,0.380592,0.500000,55900.0,84250.0,NaN,290600,0
2,test_2,4,8,0.000000,0.000000,0.600000,0.400000,0.000000,0,2.500000,...,90000.0,2.0,45000.000000,0.148270,0.400000,52000.0,96000.0,NaN,607000,0
3,test_3,28,16,0.031250,0.062500,0.468750,0.343750,0.125000,3,2.000000,...,1096000.0,13.0,84307.692308,0.475335,0.406250,18400.0,153250.0,380000.0,2305740,1
4,test_4,19,29,0.476190,0.380952,0.285714,0.238095,0.095238,6,2.333333,...,39000.0,1.0,39000.000000,0.024862,0.047619,27000.0,125100.0,270000.0,1568690,1


In [165]:
# # code_test
# agg_list = [
#         ('vip', lambda x: pd.Series(x.sum()).sort_values(ascending = False).iloc[:vip_num]),
#         ('사분위수', 'quantile')
#     ]

# train_tr.groupby('ID')["구매가격"].agg(agg_list).reset_index()

In [166]:
# # code_acting
# vip_num = int(train_tr.groupby('ID')['구매가격'].mean().count() * 0.1)
# agg_list = [
#         ('vip', lambda x: pd.Series(x.sum()).sort_values(ascending = False).iloc[:vip_num]),
#     ]

# tmp = train_tr.groupby('ID')["구매가격"].agg(agg_list).reset_index()
# train_ft = train_ft.merge(tmp, how='left')
# train_ft.head()

In [167]:
# tmp = test_tr.groupby('ID')["구매가격"].agg(agg_list).reset_index()
# test_ft = test_ft.merge(tmp, how='left')
# test_ft.head()

In [168]:
train_ft.shape, test_ft.shape

((14940, 525), (12225, 525))

In [169]:
# train_tr['중분류'].unique()

In [170]:
# train_tr['대분류'].unique()

In [171]:
# train_tr[train_tr['대분류'] == '']['중분류'].unique()

# 항상 확인하기
- 학습데이터와 테스트 데이터의 피처개수는 동일해야 함

In [172]:
train_ft.shape, test_ft.shape

((14940, 525), (12225, 525))

In [173]:
train_ft = train_ft.fillna(0)
test_ft = test_ft.fillna(0)

train_ft.isnull().sum().sum(), test_ft.isnull().sum().sum()

(0, 0)

In [174]:
train_ft.shape, test_ft.shape

((14940, 525), (12225, 525))

In [175]:
train_ft = train_ft.drop(columns="ID")
test_ft = test_ft.drop(columns="ID")

In [176]:
cols = train_ft.select_dtypes("object").columns.tolist()
cols

['주구매지점', '주구매_중분류', '주구매_대분류']

In [177]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown="ignore")

In [178]:
enc.fit_transform(train_ft[cols]).toarray()

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.]])

In [179]:
tmp = pd.DataFrame(
    enc.fit_transform(train_ft[cols]).toarray(),
    columns =  enc.get_feature_names_out()
)

train_ft = pd.concat([train_ft,tmp],axis=1).drop(columns=cols)
train_ft.head()

,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,거래개월수,...,주구매_대분류_여성캐주얼,주구매_대분류_여성캐쥬얼,주구매_대분류_영라이브,주구매_대분류_영어덜트캐쥬얼,주구매_대분류_영캐릭터,주구매_대분류_영플라자,주구매_대분류_잡화,주구매_대분류_잡화파트,주구매_대분류_케주얼_구두_아동,주구매_대분류_패션잡화
0,16,23,0.250000,0.050000,0.250000,0.400000,0.300000,3,1.818182,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,41,16,0.023810,0.357143,0.166667,0.357143,0.119048,3,2.000000,11,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,102,6,0.210526,0.464912,0.140351,0.175439,0.219298,0,2.035714,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,191,3,0.189573,0.379147,0.180095,0.236967,0.203791,3,2.293478,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,55,11,0.258065,0.112903,0.612903,0.209677,0.064516,4,2.296296,10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [180]:
test_ft.columns

Index(['내점일수', '구매주기', '주말방문비율', '봄_구매비율', '여름_구매비율', '가을_구매비율', '겨울_구매비율',
       '주구매요일', '일별평균구매건수', '거래개월수',
       ...
       '잡화관련구매비용', '잡화관련구매건수', '잡화관련구매평균', '잡화관련구매비용비율', '잡화관련구매건수비율',
       '구매가격_사분위수_Q1', '구매가격_사분위수_Q3', '상위10%_구매가격', '구매가격_합계', '상위10%_여부'],
      dtype='object', length=524)

In [181]:
tmp = pd.DataFrame(
    enc.transform(test_ft[cols]).toarray(),
    columns = enc.get_feature_names_out()
)
test_ft = pd.concat([test_ft,tmp],axis=1).drop(columns=cols)
test_ft.head()

,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,거래개월수,...,주구매_대분류_여성캐주얼,주구매_대분류_여성캐쥬얼,주구매_대분류_영라이브,주구매_대분류_영어덜트캐쥬얼,주구매_대분류_영캐릭터,주구매_대분류_영플라자,주구매_대분류_잡화,주구매_대분류_잡화파트,주구매_대분류_케주얼_구두_아동,주구매_대분류_패션잡화
0,7,32,0.571429,0.285714,0.285714,0.428571,0.000000,1,1.400000,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4,54,0.250000,0.000000,0.000000,0.750000,0.250000,0,1.333333,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4,8,0.000000,0.000000,0.600000,0.400000,0.000000,0,2.500000,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,28,16,0.031250,0.062500,0.468750,0.343750,0.125000,3,2.000000,7,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,19,29,0.476190,0.380952,0.285714,0.238095,0.095238,6,2.333333,4,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [182]:
train_ft.shape, test_ft.shape

((14940, 799), (12225, 799))

In [183]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [184]:
train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)
train_ft.head()

,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,거래개월수,...,주구매_대분류_여성캐주얼,주구매_대분류_여성캐쥬얼,주구매_대분류_영라이브,주구매_대분류_영어덜트캐쥬얼,주구매_대분류_영캐릭터,주구매_대분류_영플라자,주구매_대분류_잡화,주구매_대분류_잡화파트,주구매_대분류_케주얼_구두_아동,주구매_대분류_패션잡화
0,-0.437126,0.002987,0.257728,-1.029777,0.001191,0.838272,0.338186,0.109631,-0.057297,0.017620,...,-0.165008,-0.08084,-0.133603,-0.081258,-0.101046,-0.235019,-0.228606,-0.342432,-0.280675,-0.319269
1,0.239394,-0.356452,-1.008554,0.323951,-0.390607,0.620171,-0.552996,0.109631,0.222706,1.176010,...,-0.165008,-0.08084,-0.133603,-0.081258,-0.101046,4.254981,-0.228606,-0.342432,-0.280675,-0.319269
2,1.890101,-0.869935,0.036742,0.798943,-0.514333,-0.304527,-0.059266,-1.643370,0.277707,1.465608,...,-0.165008,-0.08084,-0.133603,-0.081258,-0.101046,-0.235019,-0.228606,-0.342432,-0.280675,-0.319269
3,4.298510,-1.023980,-0.080558,0.420933,-0.327474,0.008592,-0.135636,0.109631,0.674668,1.465608,...,-0.165008,-0.08084,-0.133603,-0.081258,-0.101046,-0.235019,-0.228606,-0.342432,3.562844,-0.319269
4,0.618244,-0.613193,0.302875,-0.752532,1.707410,-0.130285,-0.821561,0.693965,0.679008,0.886413,...,-0.165008,-0.08084,-0.133603,-0.081258,-0.101046,-0.235019,-0.228606,-0.342432,-0.280675,-0.319269


In [185]:
target = train_target["target"]
target

,target
0,1.0
1,1.0
2,0.0
3,0.0
4,0.0
...,...
14935,0.0
14936,0.0
14937,0.0
14938,1.0


In [186]:
from sklearn.model_selection import cross_val_score, KFold
cv = KFold(5, shuffle = True, random_state = 42)

In [187]:
from lightgbm import LGBMClassifier
model = LGBMClassifier(random_state=42)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [188]:
scores = cross_val_score(model, train_ft, target, cv = cv, scoring = 'f1_macro', n_jobs = -1)
scores.mean()

0.7082183197170817

In [189]:
model.fit(train_ft,target)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 5874, number of negative: 9066
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.060878 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 46327
[LightGBM] [Info] Number of data points in the train set: 14940, number of used features: 613
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.393173 -> initscore=-0.433995
[LightGBM] [Info] Start training from score -0.433995


LGBMClassifier(random_state=42)

In [190]:
pred = model.predict(test_ft)
pred

array([0., 0., 0., ..., 0., 0., 0.])

In [191]:
submit["target"] = pred
submit

,ID,target
0,test_0,0.0
1,test_1,0.0
2,test_2,0.0
3,test_3,1.0
4,test_4,1.0
...,...,...
12220,test_12220,1.0
12221,test_12221,0.0
12222,test_12222,0.0
12223,test_12223,0.0


# 추출한 피처 저장하기

In [192]:
final_project = '/content/drive/MyDrive/멋사_final_project/'

In [193]:
# train_ft.to_csv(f"{final_project}train_common_DK_2024_10_30_1차.csv",index=False)
# test_ft.to_csv(f"{final_project}test_common_DK_2024_10_30_1차.csv",index=False)

In [194]:
submit.to_csv(f"{final_project}DK_2024_11_04_16차_submit.csv",index=False)